## Feature Construction

In [1]:
# Import necessary libraries
import os
from sklearn.impute import KNNImputer
from general_used_functions import *
from collections import defaultdict
import pandas as pd
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')

In [2]:
# Load config data
config_data = load_config_file()

# Load the data from the data directory
DATA_DIR = os.getcwd() + '/data'
training_stock_df = defaultdict(list)
testing_stock_df = defaultdict(list)
stock_list = config_data['stock_dict'].keys()

# Load traininig data
for stock in stock_list:
    training_stock_data = pd.read_excel(f"{DATA_DIR}/stock_price_data/training/{stock}_stock_price_data(training).xlsx")
    training_stock_df[stock] = training_stock_data

# Load testing data
for stock in stock_list:
    testing_stock_data = pd.read_excel(f"{DATA_DIR}/stock_price_data/testing/{stock}_stock_price_data(testing).xlsx")
    testing_stock_df[stock] = testing_stock_data

In [3]:
# Initialize feature dataframe and store the date as the first column
training_feature_df = {}
testing_feature_df = {}

# Training
for stock in stock_list:
    training_feature_df[stock] = pd.DataFrame()
    training_feature_df[stock]['date'] = training_stock_df[stock]['date']

# Testing
for stock in stock_list:
    testing_feature_df[stock] = pd.DataFrame()
    testing_feature_df[stock]['date'] = testing_stock_df[stock]['date']

#### Feature 1 - 3: Exponential Weighted Moving (EMW) with halflives(days) = 5, 10 and 21  (Self-Constructed)

In [4]:
def exponential_moving_average(data, halflives):
    return data.ewm(halflife=halflives).mean()

# Training
for stock in stock_list:
    for halflife in [5, 10, 21]:
        training_feature_df[stock][f'{stock}_exponential_moving_average_{halflife}'] = exponential_moving_average(training_stock_df[stock][stock], halflife)

        if check_weird_data(training_feature_df[stock][f'{stock}_exponential_moving_average_{halflife}']):
            print(f"Feature {stock}_exponential_moving_average_{halflife} contains weird data in training data")

# Testing
for stock in stock_list:
    for halflife in [5, 10, 21]:
        testing_feature_df[stock][f'{stock}_exponential_moving_average_{halflife}'] = exponential_moving_average(testing_stock_df[stock][stock], halflife)

        if check_weird_data(testing_feature_df[stock][f'{stock}_exponential_moving_average_{halflife}']):
            print(f"Feature {stock}_exponential_moving_average_{halflife} contains weird data in testing data")

In [5]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

        date  AAPL_exponential_moving_average_5  AAPL_exponential_moving_average_10  AAPL_exponential_moving_average_21
0 2016-07-01                          23.972500                           23.972500                           23.972500
1 2016-07-05                          23.852214                           23.856102                           23.858143
2 2016-07-06                          23.863737                           23.865518                           23.866531
3 2016-07-07                          23.900616                           23.898562                           23.897631
4 2016-07-08                          23.970359                           23.960624                           23.955759
        date  AAPL_exponential_moving_average_5  AAPL_exponential_moving_average_10  AAPL_exponential_moving_average_21
0 2023-01-03                         125.070000                          125.070000                          125.070000
1 2023-01-04                         125

#### Feature 4 - 6: EWM downside deviation (DD) in log scale with halflives(days) = 5, 10, 21 (Self-Constructed)

In [6]:
def weighted_moving_downside_deviation_with_log(data, halflives, TRADING_DAYS=252):
    log_returns = np.log(data / data.shift(1))
    downside_returns = log_returns[log_returns < 0].fillna(0)
    ewm_std = downside_returns.ewm(halflife=halflives).std()
    annualized_ewm_std = ewm_std * np.sqrt(TRADING_DAYS)
    return annualized_ewm_std

# Initialize a dictionary to store weird columns for each stock
training_weird_columns_dict = {stock: [] for stock in stock_list}
testing_weird_columns_dict = {stock: [] for stock in stock_list}

# Training
for stock in stock_list:
    for halflife in [5, 10, 21]:
        downside_deviation = weighted_moving_downside_deviation_with_log(training_stock_df[stock][stock], halflife)
        downside_deviation = downside_deviation.reindex(training_stock_df[stock].index) 
        training_feature_df[stock][f'{stock}_downside_deviation_{halflife}'] = downside_deviation

        if check_weird_data(training_feature_df[stock][f'{stock}_downside_deviation_{halflife}']):
            print(f"Feature {stock}_downside_deviation_{halflife} contains weird data")
            training_weird_columns_dict[stock].append(f'{stock}_downside_deviation_{halflife}')

# Testing
for stock in stock_list:
    for halflife in [5, 10, 21]:
        downside_deviation = weighted_moving_downside_deviation_with_log(testing_stock_df[stock][stock], halflife)
        downside_deviation = downside_deviation.reindex(testing_stock_df[stock].index) 
        testing_feature_df[stock][f'{stock}_downside_deviation_{halflife}'] = downside_deviation

        if check_weird_data(testing_feature_df[stock][f'{stock}_downside_deviation_{halflife}']):
            print(f"Feature {stock}_downside_deviation_{halflife} contains weird data")
            testing_weird_columns_dict[stock].append(f'{stock}_downside_deviation_{halflife}')

Feature AMZN_downside_deviation_5 contains weird data
Feature AMZN_downside_deviation_10 contains weird data
Feature AMZN_downside_deviation_21 contains weird data
Feature GOOGL_downside_deviation_5 contains weird data
Feature GOOGL_downside_deviation_10 contains weird data
Feature GOOGL_downside_deviation_21 contains weird data
Feature MSFT_downside_deviation_5 contains weird data
Feature MSFT_downside_deviation_10 contains weird data
Feature MSFT_downside_deviation_21 contains weird data
Feature NVDA_downside_deviation_5 contains weird data
Feature NVDA_downside_deviation_10 contains weird data
Feature NVDA_downside_deviation_21 contains weird data
Feature AAPL_downside_deviation_5 contains weird data
Feature AAPL_downside_deviation_10 contains weird data
Feature AAPL_downside_deviation_21 contains weird data
Feature NFLX_downside_deviation_5 contains weird data
Feature NFLX_downside_deviation_10 contains weird data
Feature NFLX_downside_deviation_21 contains weird data
Feature AVGO_

In [7]:
# handle the weird data with KNN imputer
def handle_weird_data_with_knn_imputer(data, weird_columns):
    imputer = KNNImputer(n_neighbors=5)
    data[weird_columns] = imputer.fit_transform(data[weird_columns])
    return data


# Training
for stock, weird_columns in training_weird_columns_dict.items():
    if weird_columns:
        training_feature_df[stock] = handle_weird_data_with_knn_imputer(training_feature_df[stock], weird_columns)

# Testing
for stock, weird_columns in testing_weird_columns_dict.items():
    if weird_columns:
        testing_feature_df[stock] = handle_weird_data_with_knn_imputer(testing_feature_df[stock], weird_columns)

In [8]:
# Check the data again
for stock, stock_data in training_feature_df.items():
    if check_weird_data(stock_data):
        print(f"The {stock} stock price data still contains weird data in training data")

for stock, stock_data in testing_feature_df.items():
    if check_weird_data(stock_data):
        print(f"The {stock} stock price data still contains weird data in testing data")

In [9]:
# Clear weird_columns_dict, only clear the value, not the key
for stock in stock_list:
    training_weird_columns_dict[stock] = []

for stock in stock_list:
    testing_weird_columns_dict[stock] = []

In [10]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

        date  AAPL_exponential_moving_average_5  AAPL_exponential_moving_average_10  AAPL_exponential_moving_average_21  AAPL_downside_deviation_5  AAPL_downside_deviation_10  AAPL_downside_deviation_21
0 2016-07-01                          23.972500                           23.972500                           23.972500                   0.184606                    0.193321                    0.200102
1 2016-07-05                          23.852214                           23.856102                           23.858143                   0.184606                    0.193321                    0.200102
2 2016-07-06                          23.863737                           23.865518                           23.866531                   0.184606                    0.193321                    0.200102
3 2016-07-07                          23.900616                           23.898562                           23.897631                   0.184606                    0.193321              

#### Feature 7 - 9: Sortino ratio (Self-Constructed)

In [11]:
# First, calculate the daily return of the stock
def daily_return(stock_price):
    return stock_price.pct_change()

# Training
for stock, stock_data in training_stock_df.items():
    stock_data[f'{stock}_daily_return'] = daily_return(stock_data[stock])
    if check_weird_data(stock_data[f'{stock}_daily_return']):
            print(f"{stock} daily return contains weird data in training data")
            training_weird_columns_dict[stock].append(f'{stock}_daily_return')

# Testing
for stock, stock_data in testing_stock_df.items():
    stock_data[f'{stock}_daily_return'] = daily_return(stock_data[stock])
    if check_weird_data(stock_data[f'{stock}_daily_return']):
            print(f"{stock} daily return contains weird data in testing data")
            testing_weird_columns_dict[stock].append(f'{stock}_daily_return')

AMZN daily return contains weird data in training data
GOOGL daily return contains weird data in training data
MSFT daily return contains weird data in training data
NVDA daily return contains weird data in training data
AAPL daily return contains weird data in training data
NFLX daily return contains weird data in training data
AVGO daily return contains weird data in training data
TSLA daily return contains weird data in training data
META daily return contains weird data in training data
AMZN daily return contains weird data in testing data
GOOGL daily return contains weird data in testing data
MSFT daily return contains weird data in testing data
NVDA daily return contains weird data in testing data
AAPL daily return contains weird data in testing data
NFLX daily return contains weird data in testing data
AVGO daily return contains weird data in testing data
TSLA daily return contains weird data in testing data
META daily return contains weird data in testing data


In [12]:
# Handle the weird data with KNN imputer
# Training
for stock, weird_columns in training_weird_columns_dict.items():
    if weird_columns:
        training_stock_df[stock] = handle_weird_data_with_knn_imputer(training_stock_df[stock], weird_columns)

# Testing
for stock, weird_columns in testing_weird_columns_dict.items():
    if weird_columns:
        testing_stock_df[stock] = handle_weird_data_with_knn_imputer(testing_stock_df[stock], weird_columns)

In [13]:
# Check the data again
# Training
for stock, stock_data in training_stock_df.items():
    if check_weird_data(stock_data):
        print(f"The {stock} stock price data still contains weird data in training data")

# Testing
for stock, stock_data in testing_stock_df.items():
    if check_weird_data(stock_data):
        print(f"The {stock} stock price data still contains weird data in testing data")

In [14]:
# Clear weird_columns_dict
for stock in stock_list: 
    training_weird_columns_dict[stock] = []
    testing_weird_columns_dict[stock] = []

In [15]:
# Before calculating the Sortino ratio, 
# we need to append the daily risk free rate and 
# downside deviation to the stock data

daily_risk_free_rate = pd.read_excel(f"{DATA_DIR}/risk_free_rate/(Fama-French 3 Factors Plus Momentum - Daily Frequency)Risk-Free_Return_Rate.xlsx")
daily_risk_free_rate.rename(columns={'Risk-Free Return Rate (One Month Treasury Bill Rate)': 'daily_risk_free_rate'}, inplace=True)

# Handle unmatched date format
def parse_dates(date_series):
    return pd.to_datetime(date_series, dayfirst=False)

daily_risk_free_rate['date'] = parse_dates(daily_risk_free_rate['date'])

for stock, stock_data in training_stock_df.items():
    training_stock_df[stock] = pd.merge(stock_data, daily_risk_free_rate, on='date', how='inner')
    for halflife in [5, 10, 21]:
        downside_deviation = training_feature_df[stock][['date', f'{stock}_downside_deviation_{halflife}']]
        training_stock_df[stock] = pd.merge(training_stock_df[stock], downside_deviation, on='date', how='inner')

for stock, stock_data in testing_stock_df.items():
    testing_stock_df[stock] = pd.merge(stock_data, daily_risk_free_rate, on='date', how='inner')
    for halflife in [5, 10, 21]:
        downside_deviation = testing_feature_df[stock][['date', f'{stock}_downside_deviation_{halflife}']]
        testing_stock_df[stock] = pd.merge(testing_stock_df[stock], downside_deviation, on='date', how='inner')

In [16]:
# Calculating the Sortino ratio
def sortino_ratio(daily_return, risk_free_rate, downside_deviation):
    return (daily_return - risk_free_rate) / downside_deviation

# Training
for stock, stock_data in training_stock_df.items():
    for halflife in [5, 10, 21]:
        daily_return = stock_data[f'{stock}_daily_return']
        risk_free_rate = stock_data['daily_risk_free_rate']
        downside_deviation = stock_data[f'{stock}_downside_deviation_{halflife}']
        training_feature_df[stock][f'{stock}_sortino_ratio_{halflife}'] = sortino_ratio(daily_return, risk_free_rate, downside_deviation)

        if check_weird_data(training_feature_df[stock][f'{stock}_sortino_ratio_{halflife}']):
            print(f"{stock} sortino ratio {halflife} contains weird data in training data")
            training_weird_columns_dict[stock].append(f'{stock}_sortino_ratio_{halflife}')

# Testing
for stock, stock_data in testing_stock_df.items():
    for halflife in [5, 10, 21]:
        daily_return = stock_data[f'{stock}_daily_return']
        risk_free_rate = stock_data['daily_risk_free_rate']
        downside_deviation = stock_data[f'{stock}_downside_deviation_{halflife}']
        testing_feature_df[stock][f'{stock}_sortino_ratio_{halflife}'] = sortino_ratio(daily_return, risk_free_rate, downside_deviation)

        if check_weird_data(testing_feature_df[stock][f'{stock}_sortino_ratio_{halflife}']):
            print(f"{stock} sortino ratio {halflife} contains weird data in testing data")
            testing_weird_columns_dict[stock].append(f'{stock}_sortino_ratio_{halflife}')

In [17]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

        date  AAPL_exponential_moving_average_5  AAPL_exponential_moving_average_10  AAPL_exponential_moving_average_21  AAPL_downside_deviation_5  AAPL_downside_deviation_10  AAPL_downside_deviation_21  AAPL_sortino_ratio_5  AAPL_sortino_ratio_10  AAPL_sortino_ratio_21
0 2016-07-01                          23.972500                           23.972500                           23.972500                   0.184606                    0.193321                    0.200102              0.006554               0.006258               0.006046
1 2016-07-05                          23.852214                           23.856102                           23.858143                   0.184606                    0.193321                    0.200102             -0.050896              -0.048602              -0.046955
2 2016-07-06                          23.863737                           23.865518                           23.866531                   0.184606                    0.193321             

In [18]:
print(testing_stock_df['AAPL'].head().to_string())
print(training_stock_df['AAPL'].head().to_string())

        date        AAPL  AAPL_daily_return  daily_risk_free_rate  AAPL_downside_deviation_5  AAPL_downside_deviation_10  AAPL_downside_deviation_21
0 2023-01-03  125.070000           0.001812               0.00017                   0.110874                    0.116524                    0.117283
1 2023-01-04  126.360001           0.010314               0.00017                   0.110874                    0.116524                    0.117283
2 2023-01-05  125.019997          -0.010605               0.00017                   0.110874                    0.116524                    0.117283
3 2023-01-06  129.619995           0.036794               0.00017                   0.110874                    0.116524                    0.117283
4 2023-01-09  130.149994           0.004089               0.00017                   0.110874                    0.116524                    0.117283
        date       AAPL  AAPL_daily_return  daily_risk_free_rate  AAPL_downside_deviation_5  AAPL_downside

#### Feature 10: 12 Month Momentum (mom12m) (Self-Constructed)

In [19]:
def momentum_12m(stock_price):
    price_1m_ago = stock_price.shift(21)
    price_12m_ago = stock_price.shift(252)
    return price_1m_ago / price_12m_ago - 1

# Training
for stock, stock_data in training_stock_df.items():
    training_feature_df[stock][f'{stock}_mom12m'] = momentum_12m(stock_data[stock])
    if check_weird_data(training_feature_df[stock][f'{stock}_mom12m']):
        print(f"Feature {stock}_mom12m contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}_mom12m')

# Notice that the testing period is too short to calculate the 12-month momentum
# So it will be calculated in later stage if this feature is selected

Feature AMZN_mom12m contains weird data in training data
Feature GOOGL_mom12m contains weird data in training data
Feature MSFT_mom12m contains weird data in training data
Feature NVDA_mom12m contains weird data in training data
Feature AAPL_mom12m contains weird data in training data
Feature NFLX_mom12m contains weird data in training data
Feature AVGO_mom12m contains weird data in training data
Feature TSLA_mom12m contains weird data in training data
Feature META_mom12m contains weird data in training data


In [20]:
# Training
for stock, weird_columns in training_weird_columns_dict.items():
    if weird_columns:
        training_feature_df[stock] = handle_weird_data_with_knn_imputer(training_feature_df[stock], weird_columns)

# Check the data again
for stock, stock_data in training_feature_df.items():
    if check_weird_data(stock_data):
        print(f"The {stock} stock price data still contains weird data in training data")

# Clear weird_columns_dict
for stock in stock_list:
    training_weird_columns_dict[stock] = []

#### Feature 11: short-term reversal (mom1m) (Self-Constructed)

In [21]:
def momentum_1m(stock_price):
    stock_price_1m_ago = stock_price.shift(21)
    return stock_price / stock_price_1m_ago - 1

# Training
for stock, stock_data in training_stock_df.items():
    training_feature_df[stock][f'{stock}_mom1m'] = momentum_1m(stock_data[stock])
    if check_weird_data(training_feature_df[stock][f'{stock}_mom1m']):
        print(f"Feature {stock}_mom1m contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}_mom1m')

# Testing
for stock, stock_data in testing_stock_df.items():
    testing_feature_df[stock][f'{stock}_mom1m'] = momentum_1m(stock_data[stock])
    if check_weird_data(testing_feature_df[stock][f'{stock}_mom1m']):
        print(f"Feature {stock}_mom1m contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}_mom1m')

Feature AMZN_mom1m contains weird data in training data
Feature GOOGL_mom1m contains weird data in training data
Feature MSFT_mom1m contains weird data in training data
Feature NVDA_mom1m contains weird data in training data
Feature AAPL_mom1m contains weird data in training data
Feature NFLX_mom1m contains weird data in training data
Feature AVGO_mom1m contains weird data in training data
Feature TSLA_mom1m contains weird data in training data
Feature META_mom1m contains weird data in training data
Feature AMZN_mom1m contains weird data in testing data
Feature GOOGL_mom1m contains weird data in testing data
Feature MSFT_mom1m contains weird data in testing data
Feature NVDA_mom1m contains weird data in testing data
Feature AAPL_mom1m contains weird data in testing data
Feature NFLX_mom1m contains weird data in testing data
Feature AVGO_mom1m contains weird data in testing data
Feature TSLA_mom1m contains weird data in testing data
Feature META_mom1m contains weird data in testing data

In [22]:
# Training
for stock, weird_columns in training_weird_columns_dict.items():
    if weird_columns:
        training_feature_df[stock] = handle_weird_data_with_knn_imputer(training_feature_df[stock], weird_columns)

# Testing
for stock, weird_columns in testing_weird_columns_dict.items():
    if weird_columns:
        testing_feature_df[stock] = handle_weird_data_with_knn_imputer(testing_feature_df[stock], weird_columns)

# Check the data again
for stock, stock_data in training_feature_df.items():
    if check_weird_data(stock_data):
        print(f"The {stock} stock price data still contains weird data in training data")

for stock, stock_data in testing_feature_df.items():
    if check_weird_data(stock_data):
        print(f"The {stock} stock price data still contains weird data in testing data")

In [23]:
# Clear weird_columns_dict, only clear the value, not the key
for stock in stock_list:
    training_weird_columns_dict[stock] = []

for stock in stock_list:
    testing_weird_columns_dict[stock] = []

In [24]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

        date  AAPL_exponential_moving_average_5  AAPL_exponential_moving_average_10  AAPL_exponential_moving_average_21  AAPL_downside_deviation_5  AAPL_downside_deviation_10  AAPL_downside_deviation_21  AAPL_sortino_ratio_5  AAPL_sortino_ratio_10  AAPL_sortino_ratio_21  AAPL_mom12m  AAPL_mom1m
0 2016-07-01                          23.972500                           23.972500                           23.972500                   0.184606                    0.193321                    0.200102              0.006554               0.006258               0.006046     0.347234    0.026148
1 2016-07-05                          23.852214                           23.856102                           23.858143                   0.184606                    0.193321                    0.200102             -0.050896              -0.048602              -0.046955     0.347234    0.026148
2 2016-07-06                          23.863737                           23.865518                           23

#### Feature 12: 6-month momentum (mom6m) (Self-Constructed)

In [25]:
def momentum_6m(stock_price):
    stock_price_6m_ago = stock_price.shift(126)
    return stock_price / stock_price_6m_ago - 1

# Training
for stock, stock_data in training_stock_df.items():
    training_feature_df[stock][f'{stock}_mom6m'] = momentum_6m(stock_data[stock])
    if check_weird_data(training_feature_df[stock][f'{stock}_mom6m']):
        print(f"Feature {stock}_mom6m contains weird data")
        training_weird_columns_dict[stock].append(f'{stock}_mom6m')

# Testing
for stock, stock_data in testing_stock_df.items():
    testing_feature_df[stock][f'{stock}_mom6m'] = momentum_6m(stock_data[stock])
    if check_weird_data(testing_feature_df[stock][f'{stock}_mom6m']):
        print(f"Feature {stock}_mom6m contains weird data")
        testing_weird_columns_dict[stock].append(f'{stock}_mom6m')

Feature AMZN_mom6m contains weird data
Feature GOOGL_mom6m contains weird data
Feature MSFT_mom6m contains weird data
Feature NVDA_mom6m contains weird data
Feature AAPL_mom6m contains weird data
Feature NFLX_mom6m contains weird data
Feature AVGO_mom6m contains weird data
Feature TSLA_mom6m contains weird data
Feature META_mom6m contains weird data
Feature AMZN_mom6m contains weird data
Feature GOOGL_mom6m contains weird data
Feature MSFT_mom6m contains weird data
Feature NVDA_mom6m contains weird data
Feature AAPL_mom6m contains weird data
Feature NFLX_mom6m contains weird data
Feature AVGO_mom6m contains weird data
Feature TSLA_mom6m contains weird data
Feature META_mom6m contains weird data


In [26]:
# Training
for stock, weird_columns in training_weird_columns_dict.items():
    if weird_columns:
        training_feature_df[stock] = handle_weird_data_with_knn_imputer(training_feature_df[stock], weird_columns)

# Testing
for stock, weird_columns in testing_weird_columns_dict.items():
    if weird_columns:
        testing_feature_df[stock] = handle_weird_data_with_knn_imputer(testing_feature_df[stock], weird_columns)

# Check the data again
for stock, stock_data in training_feature_df.items():
    if check_weird_data(stock_data):
        print(f"The {stock} stock price data still contains weird data in training data")

for stock, stock_data in testing_feature_df.items():
    if check_weird_data(stock_data):
        print(f"The {stock} stock price data still contains weird data in testing data")

In [27]:
# Clear weird_columns_dict, only clear the value, not the key
for stock in stock_list:
    training_weird_columns_dict[stock] = []

for stock in stock_list:
    testing_weird_columns_dict[stock] = []

# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

        date  AAPL_exponential_moving_average_5  AAPL_exponential_moving_average_10  AAPL_exponential_moving_average_21  AAPL_downside_deviation_5  AAPL_downside_deviation_10  AAPL_downside_deviation_21  AAPL_sortino_ratio_5  AAPL_sortino_ratio_10  AAPL_sortino_ratio_21  AAPL_mom12m  AAPL_mom1m  AAPL_mom6m
0 2016-07-01                          23.972500                           23.972500                           23.972500                   0.184606                    0.193321                    0.200102              0.006554               0.006258               0.006046     0.347234    0.026148    0.169584
1 2016-07-05                          23.852214                           23.856102                           23.858143                   0.184606                    0.193321                    0.200102             -0.050896              -0.048602              -0.046955     0.347234    0.026148    0.169584
2 2016-07-06                          23.863737                           23

#### Feature 14: Log Market Equity (mvel1)

In [28]:
'''
Notice that we already have the daily price data,
and I found shareoutstanding data can be found on wrds, shrout is the number of shares outstanding
https://wrds-www.wharton.upenn.edu/pages/get-data/center-research-security-prices-crsp/annual-update/stock-events/share-outstanding/
PERMNO is the unique identifier for each stock
AMZN: 84788
GOOGL: 90319
MSFT: 10107
NVDA: 86580
AAPL: 14593
NFLX: 89393
AVGO: 93002
TSLA: 93436
META: 13407
'''
permno_dict = config_data['permno_dict']
def share_outstanding_preprocessing():
    # Load the share outstanding data
    share_outstanding_data = pd.read_excel(f"{DATA_DIR}/company_fundamentals/{stock}_share_outstanding_raw.xlsx")

    # Make sure the downloaded data is correct
    if int(permno_dict[stock]) != int(share_outstanding_data['PERMNO'][0]):
        print("You downloaded the wrong data for {stock}")
        exit()

    # rename the column from Shares Observation Date to date
    share_outstanding_data = share_outstanding_data.rename(columns={'Shares Observation Date': 'date'})
    # Convert the date column to datetime
    share_outstanding_data['date'] =  parse_dates(share_outstanding_data['date'])
    # Drop PERMNO column
    share_outstanding_data = share_outstanding_data.drop(columns=['PERMNO'])
    # Notice that the original data is expressed in thousand, so need to multiple the data by 1000
    share_outstanding_data['Shares Outstanding'] *= 1000

    return share_outstanding_data

def mvel1(stock_price, share_outstanding):
    return np.log(stock_price * share_outstanding + 1e-9)

# Share outstanding data (Training)
for stock, stock_data in training_stock_df.items():
    share_outstanding_data = share_outstanding_preprocessing()
    # Merge the share outstanding data with the stock price data (Training)
    training_date = parse_dates(training_stock_df[stock]['date'])
    training_share_outstanding = pd.merge(training_date, share_outstanding_data, on='date', how='outer')

    # Backfill the share outstanding data
    training_share_outstanding = training_share_outstanding.bfill()

    training_feature_df[stock][f'{stock}_mvel1'] = mvel1(stock_data[stock], training_share_outstanding['Shares Outstanding'])

    if check_weird_data(training_feature_df[stock][f'{stock}_mvel1']):
        print(f"Feature {stock}_mvel1 contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}_mvel1')

# Share outstanding data (Testing)
for stock, stock_data in testing_stock_df.items():
    # Merge the share outstanding data with the stock price data (Testing)
    testing_date = parse_dates(testing_stock_df[stock]['date'])
    testing_share_outstanding = pd.merge(testing_date, share_outstanding_data, on='date', how='outer')

    # Backfill the share outstanding data
    testing_share_outstanding = testing_share_outstanding.bfill()

    # There are still some missing data in the share outstanding data
    # So we need to forward fill the data
    testing_share_outstanding = testing_share_outstanding.ffill()

    testing_feature_df[stock][f'{stock}_mvel1'] = mvel1(stock_data[stock], testing_share_outstanding['Shares Outstanding'])

    if check_weird_data(testing_feature_df[stock][f'{stock}_mvel1']):
        print(f"Feature {stock}_mvel1 contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}_mvel1')

In [29]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

        date  AAPL_exponential_moving_average_5  AAPL_exponential_moving_average_10  AAPL_exponential_moving_average_21  AAPL_downside_deviation_5  AAPL_downside_deviation_10  AAPL_downside_deviation_21  AAPL_sortino_ratio_5  AAPL_sortino_ratio_10  AAPL_sortino_ratio_21  AAPL_mom12m  AAPL_mom1m  AAPL_mom6m  AAPL_mvel1
0 2016-07-01                          23.972500                           23.972500                           23.972500                   0.184606                    0.193321                    0.200102              0.006554               0.006258               0.006046     0.347234    0.026148    0.169584   23.804209
1 2016-07-05                          23.852214                           23.856102                           23.858143                   0.184606                    0.193321                    0.200102             -0.050896              -0.048602              -0.046955     0.347234    0.026148    0.169584   23.784508
2 2016-07-06                          23

#### Feature 15: Standard deviation of daily returns from month t-1 (retvol)

In [30]:
"""
Data Source: https://wrds-www.wharton.upenn.edu/pages/get-data/contributed-data-forms/global-factor-data/
PERMNO is the unique identifier for each stock
Amazon: 84788
Google: 90319
Microsoft: 10107
Nvidia: 86580
Apple: 14593
NFLX: 89393
AVGO: 93002
Tesla: 93436
Meta: 13407
"""
def get_rvol_21d(stock):
    global_factor = pd.read_excel(f"{DATA_DIR}/global_factor/{stock}_global_factor.xlsx")
    target_col = ['Day of last price observation (date)', 'Return volatility (rvol_21d)']
    rvol_21d = global_factor[target_col].copy()  # Use .copy() to avoid SettingWithCopyWarning

    # Rename the columns
    rvol_21d.rename(columns={'Return volatility (rvol_21d)': f'{stock}_rvol_21d'}, inplace=True)

    # Parse the date column
    rvol_21d['Day of last price observation (date)'] = parse_dates(rvol_21d['Day of last price observation (date)'])

    return rvol_21d

# Training
for stock, stock_data in training_stock_df.items():
    rvol_21d = get_rvol_21d(stock)

    # Merge the rvol_21d data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], rvol_21d, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values 
    training_feature_df[stock][f'{stock}_rvol_21d'] = training_feature_df[stock][f'{stock}_rvol_21d'].bfill()

    if check_weird_data(training_feature_df[stock][f'{stock}_rvol_21d']):
        print(f"Feature f'{stock}_rvol_21d' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}_rvol_21d')

# Testing
for stock, stock_data in testing_stock_df.items():
    rvol_21d = get_rvol_21d(stock)

    # Merge the rvol_21d data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], rvol_21d, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    testing_feature_df[stock][f'{stock}_rvol_21d'] = testing_feature_df[stock][f'{stock}_rvol_21d'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{stock}_rvol_21d']):
        print(f"Feature f'{stock}_rvol_21d' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}_rvol_21d')

In [31]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

        date  AAPL_exponential_moving_average_5  AAPL_exponential_moving_average_10  AAPL_exponential_moving_average_21  AAPL_downside_deviation_5  AAPL_downside_deviation_10  AAPL_downside_deviation_21  AAPL_sortino_ratio_5  AAPL_sortino_ratio_10  AAPL_sortino_ratio_21  AAPL_mom12m  AAPL_mom1m  AAPL_mom6m  AAPL_mvel1  AAPL_rvol_21d
0 2016-07-01                          23.972500                           23.972500                           23.972500                   0.184606                    0.193321                    0.200102              0.006554               0.006258               0.006046     0.347234    0.026148    0.169584   23.804209       0.016383
1 2016-07-05                          23.852214                           23.856102                           23.858143                   0.184606                    0.193321                    0.200102             -0.050896              -0.048602              -0.046955     0.347234    0.026148    0.169584   23.784508       0.01

#### Feature 16-18: Change in Shares - 1 Month (chcsho_1m, chcsho_3m, chcsho_6m)

In [32]:
def get_chcsho(stock):
    global_factor = pd.read_excel(f"{DATA_DIR}/global_factor/{stock}_global_factor.xlsx")
    target_cols = ['Day of last price observation (date)', 'Change in Shares - 1 Month (chcsho_1m)', 'Change in Shares - 3 Month (chcsho_3m)', 'Change in Shares - 6 Month (chcsho_6m)']
    chcsho = global_factor[target_cols].copy()  # Use .copy() to avoid SettingWithCopyWarning

    # Rename the columns
    chcsho.rename(columns={'Change in Shares - 1 Month (chcsho_1m)': f'{stock}_chcsho_1m', 'Change in Shares - 3 Month (chcsho_3m)': f'{stock}_chcsho_3m', 'Change in Shares - 6 Month (chcsho_6m)': f'{stock}_chcsho_6m'}, inplace=True)

    # Parse the date column
    chcsho['Day of last price observation (date)'] = parse_dates(chcsho['Day of last price observation (date)'])

    return chcsho


# Training
for stock, stock_data in training_stock_df.items():
    chcsho = get_chcsho(stock)

    # Merge the chcsho data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], chcsho, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    training_feature_df[stock][f'{stock}_chcsho_1m'] = training_feature_df[stock][f'{stock}_chcsho_1m'].bfill()
    training_feature_df[stock][f'{stock}_chcsho_3m'] = training_feature_df[stock][f'{stock}_chcsho_3m'].bfill()
    training_feature_df[stock][f'{stock}_chcsho_6m'] = training_feature_df[stock][f'{stock}_chcsho_6m'].bfill()

    if check_weird_data(training_feature_df[stock][f'{stock}_chcsho_1m']):
        print(f"Feature f'{stock}_chcsho_1m' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}_chcsho_1m')

    if check_weird_data(training_feature_df[stock][f'{stock}_chcsho_3m']):
        print(f"Feature f'{stock}_chcsho_3m' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}_chcsho_3m')

    if check_weird_data(training_feature_df[stock][f'{stock}_chcsho_6m']):
        print(f"Feature f'{stock}_chcsho_6m' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}_chcsho_6m')


# Testing
for stock, stock_data in testing_stock_df.items():
    chcsho = get_chcsho(stock)

    # Merge the chcsho data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], chcsho, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    testing_feature_df[stock][f'{stock}_chcsho_1m'] = testing_feature_df[stock][f'{stock}_chcsho_1m'].bfill()
    testing_feature_df[stock][f'{stock}_chcsho_3m'] = testing_feature_df[stock][f'{stock}_chcsho_3m'].bfill()
    testing_feature_df[stock][f'{stock}_chcsho_6m'] = testing_feature_df[stock][f'{stock}_chcsho_6m'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{stock}_chcsho_1m']):
        print(f"Feature f'{stock}_chcsho_1m' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}_chcsho_1m')

    if check_weird_data(testing_feature_df[stock][f'{stock}_chcsho_3m']):
        print(f"Feature f'{stock}_chcsho_3m' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}_chcsho_3m')

    if check_weird_data(testing_feature_df[stock][f'{stock}_chcsho_6m']):
        print(f"Feature f'{stock}_chcsho_6m' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}_chcsho_6m')

In [33]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

        date  AAPL_exponential_moving_average_5  AAPL_exponential_moving_average_10  AAPL_exponential_moving_average_21  AAPL_downside_deviation_5  AAPL_downside_deviation_10  AAPL_downside_deviation_21  AAPL_sortino_ratio_5  AAPL_sortino_ratio_10  AAPL_sortino_ratio_21  AAPL_mom12m  AAPL_mom1m  AAPL_mom6m  AAPL_mvel1  AAPL_rvol_21d  AAPL_chcsho_1m  AAPL_chcsho_3m  AAPL_chcsho_6m
0 2016-07-01                          23.972500                           23.972500                           23.972500                   0.184606                    0.193321                    0.200102              0.006554               0.006258               0.006046     0.347234    0.026148    0.169584   23.804209       0.016383       -0.000876       -0.016245       -0.028161
1 2016-07-05                          23.852214                           23.856102                           23.858143                   0.184606                    0.193321                    0.200102             -0.050896          

#### Feature 19: Amihud illiquidity (ami_126d)

In [34]:
def get_ami_126d(stock):
    global_factor = pd.read_excel(f"{DATA_DIR}/global_factor/{stock}_global_factor.xlsx")
    target_cols = ['Day of last price observation (date)', 'Amihud Measure (ami_126d)']
    ami_126d = global_factor[target_cols].copy()
    

    # Rename the columns
    ami_126d.rename(columns={'Amihud Measure (ami_126d)': f'{stock}_ami_126d'}, inplace=True)

    # Parse the date column
    ami_126d['Day of last price observation (date)'] = parse_dates(ami_126d['Day of last price observation (date)'])

    return ami_126d

# Training
for stock, stock_data in training_stock_df.items():
    ami_126d = get_ami_126d(stock)

    # Merge the ami_126d data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], ami_126d, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    training_feature_df[stock][f'{stock}_ami_126d'] = training_feature_df[stock][f'{stock}_ami_126d'].bfill()

    if check_weird_data(training_feature_df[stock][f'{stock}_ami_126d']):
        print(f"Feature f'{stock}_ami_126d' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}_ami_126d')

# Testing
for stock, stock_data in testing_stock_df.items():
    ami_126d = get_ami_126d(stock)

    # Merge the ami_126d data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], ami_126d, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    testing_feature_df[stock][f'{stock}_ami_126d'] = testing_feature_df[stock][f'{stock}_ami_126d'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{stock}_ami_126d']):
        print(f"Feature f'{stock}_ami_126d' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}_ami_126d')

In [35]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

        date  AAPL_exponential_moving_average_5  AAPL_exponential_moving_average_10  AAPL_exponential_moving_average_21  AAPL_downside_deviation_5  AAPL_downside_deviation_10  AAPL_downside_deviation_21  AAPL_sortino_ratio_5  AAPL_sortino_ratio_10  AAPL_sortino_ratio_21  AAPL_mom12m  AAPL_mom1m  AAPL_mom6m  AAPL_mvel1  AAPL_rvol_21d  AAPL_chcsho_1m  AAPL_chcsho_3m  AAPL_chcsho_6m  AAPL_ami_126d
0 2016-07-01                          23.972500                           23.972500                           23.972500                   0.184606                    0.193321                    0.200102              0.006554               0.006258               0.006046     0.347234    0.026148    0.169584   23.804209       0.016383       -0.000876       -0.016245       -0.028161       0.000003
1 2016-07-05                          23.852214                           23.856102                           23.858143                   0.184606                    0.193321                    0.200102  

#### Feature 20: Age of the firms in months (age)

In [36]:
def get_age(stock):
    global_factor = pd.read_excel(f"{DATA_DIR}/global_factor/{stock}_global_factor.xlsx")
    target_cols = ['Day of last price observation (date)', 'Firm age (age)']
    age = global_factor[target_cols].copy()
    

    # Rename the columns
    age.rename(columns={'Firm age (age)': f'{stock}_age'}, inplace=True)

    # Parse the date column
    age['Day of last price observation (date)'] = parse_dates(age['Day of last price observation (date)'])

    return age

# Training
for stock, stock_data in training_stock_df.items():
    age = get_age(stock)

    # Merge the age data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], age, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    training_feature_df[stock][f'{stock}_age'] = training_feature_df[stock][f'{stock}_age'].bfill()

    if check_weird_data(training_feature_df[stock][f'{stock}_age']):
        print(f"Feature f'{stock}_age' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}_age')

# Testing
for stock, stock_data in testing_stock_df.items():
    age = get_age(stock)

    # Merge the age data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], age, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    testing_feature_df[stock][f'{stock}_age'] = testing_feature_df[stock][f'{stock}_age'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{stock}_age']):
        print(f"Feature f'{stock}_age' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}_age')

In [37]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

        date  AAPL_exponential_moving_average_5  AAPL_exponential_moving_average_10  AAPL_exponential_moving_average_21  AAPL_downside_deviation_5  AAPL_downside_deviation_10  AAPL_downside_deviation_21  AAPL_sortino_ratio_5  AAPL_sortino_ratio_10  AAPL_sortino_ratio_21  AAPL_mom12m  AAPL_mom1m  AAPL_mom6m  AAPL_mvel1  AAPL_rvol_21d  AAPL_chcsho_1m  AAPL_chcsho_3m  AAPL_chcsho_6m  AAPL_ami_126d  AAPL_age
0 2016-07-01                          23.972500                           23.972500                           23.972500                   0.184606                    0.193321                    0.200102              0.006554               0.006258               0.006046     0.347234    0.026148    0.169584   23.804209       0.016383       -0.000876       -0.016245       -0.028161       0.000003     439.0
1 2016-07-05                          23.852214                           23.856102                           23.858143                   0.184606                    0.193321          

#### Feature 21: Change in 6-month momentum (chmom) (Self-Constructed)

In [38]:
def get_chmom(stock_price):
    price_1m_ago = stock_price.shift(21)
    price_6m_ago = stock_price.shift(126)

    price_7m_ago = stock_price.shift(147)
    price_12m_ago = stock_price.shift(252)
    return (price_1m_ago / price_6m_ago) - 1 - ( (price_7m_ago / price_12m_ago) - 1 )

# Training
for stock, stock_data in training_stock_df.items():
    training_feature_df[stock][f'{stock}_chmom'] = get_chmom(stock_data[stock])
    if check_weird_data(training_feature_df[stock][f'{stock}_chmom']):
        print(f"Feature {stock}_chmom contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}_chmom')

# Similarly, the testing period is too short to calculate the change in 6-month momentum
# So it will be calculated in later stage if this feature is selected

Feature AMZN_chmom contains weird data in training data
Feature GOOGL_chmom contains weird data in training data
Feature MSFT_chmom contains weird data in training data
Feature NVDA_chmom contains weird data in training data
Feature AAPL_chmom contains weird data in training data
Feature NFLX_chmom contains weird data in training data
Feature AVGO_chmom contains weird data in training data
Feature TSLA_chmom contains weird data in training data
Feature META_chmom contains weird data in training data


In [39]:
# Training
for stock, weird_columns in training_weird_columns_dict.items():
    if weird_columns:
        training_feature_df[stock] = handle_weird_data_with_knn_imputer(training_feature_df[stock], weird_columns)

# Check the data again
for stock, stock_data in training_feature_df.items():
    if check_weird_data(stock_data):
        print(f"The {stock} stock price data still contains weird data in training data")

In [40]:
# Clear weird_columns_dict, only clear the value, not the key
for stock in stock_list:
    training_weird_columns_dict[stock] = []

# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

        date  AAPL_exponential_moving_average_5  AAPL_exponential_moving_average_10  AAPL_exponential_moving_average_21  AAPL_downside_deviation_5  AAPL_downside_deviation_10  AAPL_downside_deviation_21  AAPL_sortino_ratio_5  AAPL_sortino_ratio_10  AAPL_sortino_ratio_21  AAPL_mom12m  AAPL_mom1m  AAPL_mom6m  AAPL_mvel1  AAPL_rvol_21d  AAPL_chcsho_1m  AAPL_chcsho_3m  AAPL_chcsho_6m  AAPL_ami_126d  AAPL_age  AAPL_chmom
0 2016-07-01                          23.972500                           23.972500                           23.972500                   0.184606                    0.193321                    0.200102              0.006554               0.006258               0.006046     0.347234    0.026148    0.169584   23.804209       0.016383       -0.000876       -0.016245       -0.028161       0.000003     439.0   -0.025031
1 2016-07-05                          23.852214                           23.856102                           23.858143                   0.184606              

#### Feature 22: Maximum daily return (maxret) (Self-Constructed)

In [41]:
def get_maxret(stock_df, stock_ticket):
    # Create a copy to avoid SettingWithCopy issues
    df = stock_df.copy()
    df.loc[:, 'year_month'] = df['date'].dt.to_period('M')
    monthly_max = df.groupby('year_month')[f'{stock_ticket}_daily_return'].max().reset_index()
    # Rename the computed column
    monthly_max = monthly_max.rename(columns={f'{stock_ticket}_daily_return': f'{stock_ticket}_maxret'})
    # Shift the max returns down by one period so that each row corresponds to calendar month t-1
    monthly_max[f'{stock_ticket}_maxret'] = monthly_max[f'{stock_ticket}_maxret'].shift(1)
    # Convert the period back to a timestamp; here we choose the month-end timestamp
    monthly_max['year_month'] = monthly_max['year_month'].dt.to_timestamp('M')
    return monthly_max

# Training
for stock, stock_data in training_stock_df.items():
    maxret = get_maxret(stock_data, stock)
    training_feature_df[stock] = pd.merge(training_feature_df[stock], maxret, how='left', left_on='date', right_on='year_month')
    training_feature_df[stock].drop(columns=['year_month'], inplace=True)
    training_feature_df[stock][f'{stock}_maxret'] = training_feature_df[stock][f'{stock}_maxret'].bfill()

    if check_weird_data(training_feature_df[stock][f'{stock}_maxret']):
        print(f"Feature {stock}_maxret contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}_maxret')

# Testing
for stock, stock_data in testing_stock_df.items():
    maxret = get_maxret(stock_data, stock)
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], maxret, how='left', left_on='date', right_on='year_month')
    testing_feature_df[stock].drop(columns=['year_month'], inplace=True)
    testing_feature_df[stock][f'{stock}_maxret'] = testing_feature_df[stock][f'{stock}_maxret'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{stock}_maxret']):
        print(f"Feature {stock}_maxret contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}_maxret')

Feature AMZN_maxret contains weird data in training data
Feature GOOGL_maxret contains weird data in training data
Feature MSFT_maxret contains weird data in training data
Feature NVDA_maxret contains weird data in training data
Feature AAPL_maxret contains weird data in training data
Feature NFLX_maxret contains weird data in training data
Feature AVGO_maxret contains weird data in training data
Feature TSLA_maxret contains weird data in training data
Feature META_maxret contains weird data in training data
Feature AMZN_maxret contains weird data in testing data
Feature GOOGL_maxret contains weird data in testing data
Feature MSFT_maxret contains weird data in testing data
Feature NVDA_maxret contains weird data in testing data
Feature AAPL_maxret contains weird data in testing data
Feature NFLX_maxret contains weird data in testing data
Feature AVGO_maxret contains weird data in testing data
Feature TSLA_maxret contains weird data in testing data
Feature META_maxret contains weird da

In [42]:
# Training
for stock, weird_columns in training_weird_columns_dict.items():
    if weird_columns:
        training_feature_df[stock] = handle_weird_data_with_knn_imputer(training_feature_df[stock], weird_columns)

# Testing
for stock, weird_columns in testing_weird_columns_dict.items():
    if weird_columns:
        testing_feature_df[stock] = handle_weird_data_with_knn_imputer(testing_feature_df[stock], weird_columns)

# Check the data again
for stock, stock_data in training_feature_df.items():
    if check_weird_data(stock_data):
        print(f"The {stock} stock price data still contains weird data in training data")

for stock, stock_data in testing_feature_df.items():
    if check_weird_data(stock_data):
        print(f"The {stock} stock price data still contains weird data in testing data")


In [43]:
# Clear weird_columns_dict, only clear the value, not the key
for stock in stock_list:
    training_weird_columns_dict[stock] = []

for stock in stock_list:
    testing_weird_columns_dict[stock] = []

# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())


        date  AAPL_exponential_moving_average_5  AAPL_exponential_moving_average_10  AAPL_exponential_moving_average_21  AAPL_downside_deviation_5  AAPL_downside_deviation_10  AAPL_downside_deviation_21  AAPL_sortino_ratio_5  AAPL_sortino_ratio_10  AAPL_sortino_ratio_21  AAPL_mom12m  AAPL_mom1m  AAPL_mom6m  AAPL_mvel1  AAPL_rvol_21d  AAPL_chcsho_1m  AAPL_chcsho_3m  AAPL_chcsho_6m  AAPL_ami_126d  AAPL_age  AAPL_chmom  AAPL_maxret
0 2016-07-01                          23.972500                           23.972500                           23.972500                   0.184606                    0.193321                    0.200102              0.006554               0.006258               0.006046     0.347234    0.026148    0.169584   23.804209       0.016383       -0.000876       -0.016245       -0.028161       0.000003     439.0   -0.025031     0.064963
1 2016-07-05                          23.852214                           23.856102                           23.858143               

#### Feature 23: Dollar trading volume (dolvol)

In [44]:
def get_dolvol(stock):
    gobal_factor = pd.read_excel(f"{DATA_DIR}/global_factor/{stock}_global_factor.xlsx")
    target_cols = ['Day of last price observation (date)', 'Dollar trading volume (dolvol)']
    dolvol = gobal_factor[target_cols].copy()
    
    # Rename the columns
    dolvol.rename(columns={'Dollar trading volume (dolvol)': f'{stock}_dolvol'}, inplace=True)

    # Parse the date column
    dolvol['Day of last price observation (date)'] = parse_dates(dolvol['Day of last price observation (date)'])

    return dolvol

# Training
for stock, stock_data in training_stock_df.items():
    dolvol = get_dolvol(stock)

    # Merge the dolvol data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], dolvol, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    training_feature_df[stock][f'{stock}_dolvol'] = training_feature_df[stock][f'{stock}_dolvol'].bfill()

    if check_weird_data(training_feature_df[stock][f'{stock}_dolvol']):
        print(f"Feature f'{stock}_dolvol' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}_dolvol')

# Testing
for stock, stock_data in testing_stock_df.items():
    dolvol = get_dolvol(stock)

    # Merge the dolvol data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], dolvol, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    testing_feature_df[stock][f'{stock}_dolvol'] = testing_feature_df[stock][f'{stock}_dolvol'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{stock}_dolvol']):
        print(f"Feature f'{stock}_dolvol' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}_dolvol')

In [45]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

        date  AAPL_exponential_moving_average_5  AAPL_exponential_moving_average_10  AAPL_exponential_moving_average_21  AAPL_downside_deviation_5  AAPL_downside_deviation_10  AAPL_downside_deviation_21  AAPL_sortino_ratio_5  AAPL_sortino_ratio_10  AAPL_sortino_ratio_21  AAPL_mom12m  AAPL_mom1m  AAPL_mom6m  AAPL_mvel1  AAPL_rvol_21d  AAPL_chcsho_1m  AAPL_chcsho_3m  AAPL_chcsho_6m  AAPL_ami_126d  AAPL_age  AAPL_chmom  AAPL_maxret   AAPL_dolvol
0 2016-07-01                          23.972500                           23.972500                           23.972500                   0.184606                    0.193321                    0.200102              0.006554               0.006258               0.006046     0.347234    0.026148    0.169584   23.804209       0.016383       -0.000876       -0.016245       -0.028161       0.000003     439.0   -0.025031     0.064963  7.149022e+10
1 2016-07-05                          23.852214                           23.856102                       

#### Feature 24: 60 Month CAPM Beta (beta_60m)

In [46]:
def get_beta_60m(stock):
    global_factor = pd.read_excel(f"{DATA_DIR}/global_factor/{stock}_global_factor.xlsx")
    target_cols = ['Day of last price observation (date)', 'Market Beta (beta_60m)']
    beta_60m = global_factor[target_cols].copy()
    

    # Rename the columns
    beta_60m.rename(columns={'Market Beta (beta_60m)': f'{stock}_beta_60m'}, inplace=True)

    # Parse the date column
    beta_60m['Day of last price observation (date)'] = parse_dates(beta_60m['Day of last price observation (date)'])

    return beta_60m

# Training
for stock, stock_data in training_stock_df.items():
    beta_60m = get_beta_60m(stock)

    # Merge the beta_60m data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], beta_60m, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    training_feature_df[stock][f'{stock}_beta_60m'] = training_feature_df[stock][f'{stock}_beta_60m'].bfill()

    if check_weird_data(training_feature_df[stock][f'{stock}_beta_60m']):
        print(f"Feature f'{stock}_beta_60m' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}_beta_60m')

# Testing
for stock, stock_data in testing_stock_df.items():
    beta_60m = get_beta_60m(stock)

    # Merge the beta_60m data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], beta_60m, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    testing_feature_df[stock][f'{stock}_beta_60m'] = testing_feature_df[stock][f'{stock}_beta_60m'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{stock}_beta_60m']):
        print(f"Feature f'{stock}_beta_60m' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}_beta_60m')

In [47]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

        date  AAPL_exponential_moving_average_5  AAPL_exponential_moving_average_10  AAPL_exponential_moving_average_21  AAPL_downside_deviation_5  AAPL_downside_deviation_10  AAPL_downside_deviation_21  AAPL_sortino_ratio_5  AAPL_sortino_ratio_10  AAPL_sortino_ratio_21  AAPL_mom12m  AAPL_mom1m  AAPL_mom6m  AAPL_mvel1  AAPL_rvol_21d  AAPL_chcsho_1m  AAPL_chcsho_3m  AAPL_chcsho_6m  AAPL_ami_126d  AAPL_age  AAPL_chmom  AAPL_maxret   AAPL_dolvol  AAPL_beta_60m
0 2016-07-01                          23.972500                           23.972500                           23.972500                   0.184606                    0.193321                    0.200102              0.006554               0.006258               0.006046     0.347234    0.026148    0.169584   23.804209       0.016383       -0.000876       -0.016245       -0.028161       0.000003     439.0   -0.025031     0.064963  7.149022e+10       0.982327
1 2016-07-05                          23.852214                           23

#### Feature 25-27: Number of zero trades with turnover as tiebreaker (1 month, 6 months, 12 months) (zero_trades_21d, zero_trades_126d, zero_trades_252d)

In [48]:
def get_zero_trades(stock):
    global_factor = pd.read_excel(f"{DATA_DIR}/global_factor/{stock}_global_factor.xlsx")
    target_cols = ['Day of last price observation (date)', 'Number of zero trades with turnover as tiebreaker (1 month) (ze', 'Number of zero trades with turnover as tiebreaker (6 months) (z', 'Number of zero trades with turnover as tiebreaker (12 months) (' ]
    zero_trades = global_factor[target_cols].copy()
    

    # Rename the columns
    zero_trades.rename(columns={
        'Number of zero trades with turnover as tiebreaker (1 month) (ze': f'{stock}_zero_trades_1m', 
        'Number of zero trades with turnover as tiebreaker (6 months) (z': f'{stock}_zero_trades_6m', 
        'Number of zero trades with turnover as tiebreaker (12 months) (': f'{stock}_zero_trades_12m'}, inplace=True)

    # Parse the date column
    zero_trades['Day of last price observation (date)'] = parse_dates(zero_trades['Day of last price observation (date)'])

    return zero_trades

# Training
for stock, stock_data in training_stock_df.items():
    zero_trades = get_zero_trades(stock)

    # Merge the zero_trades data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], zero_trades, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    training_feature_df[stock][f'{stock}_zero_trades_1m'] = training_feature_df[stock][f'{stock}_zero_trades_1m'].bfill()
    training_feature_df[stock][f'{stock}_zero_trades_6m'] = training_feature_df[stock][f'{stock}_zero_trades_6m'].bfill()
    training_feature_df[stock][f'{stock}_zero_trades_12m'] = training_feature_df[stock][f'{stock}_zero_trades_12m'].bfill()

    if check_weird_data(training_feature_df[stock][f'{stock}_zero_trades_1m']):
        print(f"Feature f'{stock}_zero_trades_1m' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}_zero_trades_1m')

    if check_weird_data(training_feature_df[stock][f'{stock}_zero_trades_6m']):
        print(f"Feature f'{stock}_zero_trades_6m' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}_zero_trades_6m')

    if check_weird_data(training_feature_df[stock][f'{stock}_zero_trades_12m']):
        print(f"Feature f'{stock}_zero_trades_12m' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}_zero_trades_12m')

# Testing
for stock, stock_data in testing_stock_df.items():
    zero_trades = get_zero_trades(stock)

    # Merge the zero_trades data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], zero_trades, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    testing_feature_df[stock][f'{stock}_zero_trades_1m'] = testing_feature_df[stock][f'{stock}_zero_trades_1m'].bfill()
    testing_feature_df[stock][f'{stock}_zero_trades_6m'] = testing_feature_df[stock][f'{stock}_zero_trades_6m'].bfill()
    testing_feature_df[stock][f'{stock}_zero_trades_12m'] = testing_feature_df[stock][f'{stock}_zero_trades_12m'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{stock}_zero_trades_1m']):
        print(f"Feature f'{stock}_zero_trades_1m' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}_zero_trades_1m')

    if check_weird_data(testing_feature_df[stock][f'{stock}_zero_trades_6m']):
        print(f"Feature f'{stock}_zero_trades_6m' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}_zero_trades_6m')

    if check_weird_data(testing_feature_df[stock][f'{stock}_zero_trades_12m']):
        print(f"Feature f'{stock}_zero_trades_12m' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}_zero_trades_12m')


In [49]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

        date  AAPL_exponential_moving_average_5  AAPL_exponential_moving_average_10  AAPL_exponential_moving_average_21  AAPL_downside_deviation_5  AAPL_downside_deviation_10  AAPL_downside_deviation_21  AAPL_sortino_ratio_5  AAPL_sortino_ratio_10  AAPL_sortino_ratio_21  AAPL_mom12m  AAPL_mom1m  AAPL_mom6m  AAPL_mvel1  AAPL_rvol_21d  AAPL_chcsho_1m  AAPL_chcsho_3m  AAPL_chcsho_6m  AAPL_ami_126d  AAPL_age  AAPL_chmom  AAPL_maxret   AAPL_dolvol  AAPL_beta_60m  AAPL_zero_trades_1m  AAPL_zero_trades_6m  AAPL_zero_trades_12m
0 2016-07-01                          23.972500                           23.972500                           23.972500                   0.184606                    0.193321                    0.200102              0.006554               0.006258               0.006046     0.347234    0.026148    0.169584   23.804209       0.016383       -0.000876       -0.016245       -0.028161       0.000003     439.0   -0.025031     0.064963  7.149022e+10       0.982327             

#### Feature 13: Frazzini-Pedersen market beta (betabab_1260d)

In [50]:
def get_betabab_1260d(stock):
    global_factor = pd.read_excel(f"{DATA_DIR}/global_factor/{stock}_global_factor.xlsx")
    target_cols = ['Day of last price observation (date)', 'Frazzini-Pedersen market beta (betabab_1260d)']
    betabab_1260d = global_factor[target_cols].copy()
    

    # Rename the columns
    betabab_1260d.rename(columns={'Frazzini-Pedersen market beta (betabab_1260d)': f'{stock}_betabab_1260d'}, inplace=True)

    # Parse the date column
    betabab_1260d['Day of last price observation (date)'] = parse_dates(betabab_1260d['Day of last price observation (date)'])

    return betabab_1260d

# Training
for stock, stock_data in training_stock_df.items():
    betabab_1260d = get_betabab_1260d(stock)

    # Merge the data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], betabab_1260d, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    training_feature_df[stock][f'{stock}_betabab_1260d'] = training_feature_df[stock][f'{stock}_betabab_1260d'].bfill()

    if check_weird_data(training_feature_df[stock][f'{stock}_betabab_1260d']):
        print(f"Feature f'{stock}_betabab_1260d' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}_betabab_1260d')

# Testing
for stock, stock_data in testing_stock_df.items():
    betabab_1260d = get_betabab_1260d(stock)

    # Merge the data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], betabab_1260d, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    testing_feature_df[stock][f'{stock}_betabab_1260d'] = testing_feature_df[stock][f'{stock}_betabab_1260d'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{stock}_betabab_1260d']):
        print(f"Feature f'{stock}_betabab_1260d' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}_betabab_1260d')

In [51]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

        date  AAPL_exponential_moving_average_5  AAPL_exponential_moving_average_10  AAPL_exponential_moving_average_21  AAPL_downside_deviation_5  AAPL_downside_deviation_10  AAPL_downside_deviation_21  AAPL_sortino_ratio_5  AAPL_sortino_ratio_10  AAPL_sortino_ratio_21  AAPL_mom12m  AAPL_mom1m  AAPL_mom6m  AAPL_mvel1  AAPL_rvol_21d  AAPL_chcsho_1m  AAPL_chcsho_3m  AAPL_chcsho_6m  AAPL_ami_126d  AAPL_age  AAPL_chmom  AAPL_maxret   AAPL_dolvol  AAPL_beta_60m  AAPL_zero_trades_1m  AAPL_zero_trades_6m  AAPL_zero_trades_12m  AAPL_betabab_1260d
0 2016-07-01                          23.972500                           23.972500                           23.972500                   0.184606                    0.193321                    0.200102              0.006554               0.006258               0.006046     0.347234    0.026148    0.169584   23.804209       0.016383       -0.000876       -0.016245       -0.028161       0.000003     439.0   -0.025031     0.064963  7.149022e+10       0

#### Feature 26: Earnings variability (earnings_variability)

In [52]:
def get_earnings_variability(stock):
    global_factor = pd.read_excel(f"{DATA_DIR}/global_factor/{stock}_global_factor.xlsx")
    target_cols = ['Day of last price observation (date)', 'Earnings variability (earnings_variability)']
    earnings_variability = global_factor[target_cols].copy()

    # Rename the columns
    earnings_variability.rename(columns={'Earnings variability (earnings_variability)': f'{stock}_earnings_variability'}, inplace=True)

    # Parse the date column
    earnings_variability['Day of last price observation (date)'] = parse_dates(earnings_variability['Day of last price observation (date)'])

    return earnings_variability

# Training
for stock, stock_data in training_stock_df.items():
    earnings_variability = get_earnings_variability(stock)

    # Merge the data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], earnings_variability, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    training_feature_df[stock][f'{stock}_earnings_variability'] = training_feature_df[stock][f'{stock}_earnings_variability'].bfill()

    if check_weird_data(training_feature_df[stock][f'{stock}_earnings_variability']):
        print(f"Feature f'{stock}_earnings_variability' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}_earnings_variability')

# Testing
for stock, stock_data in testing_stock_df.items():
    earnings_variability = get_earnings_variability(stock)

    # Merge the data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], earnings_variability, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    testing_feature_df[stock][f'{stock}_earnings_variability'] = testing_feature_df[stock][f'{stock}_earnings_variability'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{stock}_earnings_variability']):
        print(f"Feature f'{stock}_earnings_variability' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}_earnings_variability')


In [53]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

        date  AAPL_exponential_moving_average_5  AAPL_exponential_moving_average_10  AAPL_exponential_moving_average_21  AAPL_downside_deviation_5  AAPL_downside_deviation_10  AAPL_downside_deviation_21  AAPL_sortino_ratio_5  AAPL_sortino_ratio_10  AAPL_sortino_ratio_21  AAPL_mom12m  AAPL_mom1m  AAPL_mom6m  AAPL_mvel1  AAPL_rvol_21d  AAPL_chcsho_1m  AAPL_chcsho_3m  AAPL_chcsho_6m  AAPL_ami_126d  AAPL_age  AAPL_chmom  AAPL_maxret   AAPL_dolvol  AAPL_beta_60m  AAPL_zero_trades_1m  AAPL_zero_trades_6m  AAPL_zero_trades_12m  AAPL_betabab_1260d  AAPL_earnings_variability
0 2016-07-01                          23.972500                           23.972500                           23.972500                   0.184606                    0.193321                    0.200102              0.006554               0.006258               0.006046     0.347234    0.026148    0.169584   23.804209       0.016383       -0.000876       -0.016245       -0.028161       0.000003     439.0   -0.025031     0.0

#### Feature 27: Operating Cash Flow to Sales Quarterly Volatility (ocfq_saleq_std)

In [54]:
def get_ocfq_saleq_std(stock):
    global_factor = pd.read_excel(f"{DATA_DIR}/global_factor/{stock}_global_factor.xlsx")
    target_cols = ['Day of last price observation (date)', 'Cash flow volatility (ocfq_saleq_std)']
    ocfq_saleq_std = global_factor[target_cols].copy()

    # Rename the columns
    ocfq_saleq_std.rename(columns={'Cash flow volatility (ocfq_saleq_std)': f'{stock}_ocfq_saleq_std'}, inplace=True)

    # Parse the date column
    ocfq_saleq_std['Day of last price observation (date)'] = parse_dates(ocfq_saleq_std['Day of last price observation (date)'])

    return ocfq_saleq_std

# Training
for stock, stock_data in training_stock_df.items():
    ocfq_saleq_std = get_ocfq_saleq_std(stock)

    # Merge the data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], ocfq_saleq_std, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    training_feature_df[stock][f'{stock}_ocfq_saleq_std'] = training_feature_df[stock][f'{stock}_ocfq_saleq_std'].bfill()

    if check_weird_data(training_feature_df[stock][f'{stock}_ocfq_saleq_std']):
        print(f"Feature f'{stock}_ocfq_saleq_std' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}_ocfq_saleq_std')

# Testing
for stock, stock_data in testing_stock_df.items():
    ocfq_saleq_std = get_ocfq_saleq_std(stock)

    # Merge the data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], ocfq_saleq_std, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    testing_feature_df[stock][f'{stock}_ocfq_saleq_std'] = testing_feature_df[stock][f'{stock}_ocfq_saleq_std'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{stock}_ocfq_saleq_std']):
        print(f"Feature f'{stock}_ocfq_saleq_std' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}_ocfq_saleq_std')

In [55]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

        date  AAPL_exponential_moving_average_5  AAPL_exponential_moving_average_10  AAPL_exponential_moving_average_21  AAPL_downside_deviation_5  AAPL_downside_deviation_10  AAPL_downside_deviation_21  AAPL_sortino_ratio_5  AAPL_sortino_ratio_10  AAPL_sortino_ratio_21  AAPL_mom12m  AAPL_mom1m  AAPL_mom6m  AAPL_mvel1  AAPL_rvol_21d  AAPL_chcsho_1m  AAPL_chcsho_3m  AAPL_chcsho_6m  AAPL_ami_126d  AAPL_age  AAPL_chmom  AAPL_maxret   AAPL_dolvol  AAPL_beta_60m  AAPL_zero_trades_1m  AAPL_zero_trades_6m  AAPL_zero_trades_12m  AAPL_betabab_1260d  AAPL_earnings_variability  AAPL_ocfq_saleq_std
0 2016-07-01                          23.972500                           23.972500                           23.972500                   0.184606                    0.193321                    0.200102              0.006554               0.006258               0.006046     0.347234    0.026148    0.169584   23.804209       0.016383       -0.000876       -0.016245       -0.028161       0.000003     439.

#### Feature 28: Turnover (turnover_126d)

In [56]:
def get_turnover_126d(stock):
    global_factor = pd.read_excel(f"{DATA_DIR}/global_factor/{stock}_global_factor.xlsx")
    target_cols = ['Day of last price observation (date)', 'Share turnover (turnover_126d)']
    turnover_126d = global_factor[target_cols].copy()

    # Rename the columns
    turnover_126d.rename(columns={'Share turnover (turnover_126d)': f'{stock}_turnover_126d'}, inplace=True)

    # Parse the date column
    turnover_126d['Day of last price observation (date)'] = parse_dates(turnover_126d['Day of last price observation (date)'])

    return turnover_126d

# Training
for stock, stock_data in training_stock_df.items():
    turnover_126d = get_turnover_126d(stock)

    # Merge the data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], turnover_126d, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    training_feature_df[stock][f'{stock}_turnover_126d'] = training_feature_df[stock][f'{stock}_turnover_126d'].bfill()

    if check_weird_data(training_feature_df[stock][f'{stock}_turnover_126d']):
        print(f"Feature f'{stock}_turnover_126d' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}_turnover_126d')

# Testing
for stock, stock_data in testing_stock_df.items():
    turnover_126d = get_turnover_126d(stock)

    # Merge the data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], turnover_126d, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    testing_feature_df[stock][f'{stock}_turnover_126d'] = testing_feature_df[stock][f'{stock}_turnover_126d'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{stock}_turnover_126d']):
        print(f"Feature f'{stock}_turnover_126d' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}_turnover_126d')

In [57]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

        date  AAPL_exponential_moving_average_5  AAPL_exponential_moving_average_10  AAPL_exponential_moving_average_21  AAPL_downside_deviation_5  AAPL_downside_deviation_10  AAPL_downside_deviation_21  AAPL_sortino_ratio_5  AAPL_sortino_ratio_10  AAPL_sortino_ratio_21  AAPL_mom12m  AAPL_mom1m  AAPL_mom6m  AAPL_mvel1  AAPL_rvol_21d  AAPL_chcsho_1m  AAPL_chcsho_3m  AAPL_chcsho_6m  AAPL_ami_126d  AAPL_age  AAPL_chmom  AAPL_maxret   AAPL_dolvol  AAPL_beta_60m  AAPL_zero_trades_1m  AAPL_zero_trades_6m  AAPL_zero_trades_12m  AAPL_betabab_1260d  AAPL_earnings_variability  AAPL_ocfq_saleq_std  AAPL_turnover_126d
0 2016-07-01                          23.972500                           23.972500                           23.972500                   0.184606                    0.193321                    0.200102              0.006554               0.006258               0.006046     0.347234    0.026148    0.169584   23.804209       0.016383       -0.000876       -0.016245       -0.028161    

#### Feature 29: Current price to high price over last year (prc_highprc_252d)

In [58]:
def get_prc_highprc_252d(stock):
    global_factor = pd.read_excel(f"{DATA_DIR}/global_factor/{stock}_global_factor.xlsx")
    target_cols = ['Day of last price observation (date)', 'Current price to high price over last year (prc_highprc_252d)']
    prc_highprc_252d = global_factor[target_cols].copy()

    # Rename the columns
    prc_highprc_252d.rename(columns={target_cols[1]: f'{stock}_prc_highprc_252d'}, inplace=True)

    # Parse the date column
    prc_highprc_252d[target_cols[0]] = parse_dates(prc_highprc_252d[target_cols[0]])

    return prc_highprc_252d

# Training
for stock, stock_data in training_stock_df.items():
    prc_highprc_252d = get_prc_highprc_252d(stock)

    # Merge the data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], prc_highprc_252d, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    training_feature_df[stock][f'{stock}_prc_highprc_252d'] = training_feature_df[stock][f'{stock}_prc_highprc_252d'].bfill()

    if check_weird_data(training_feature_df[stock][f'{stock}_prc_highprc_252d']):
        print(f"Feature f'{stock}_prc_highprc_252d' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}_prc_highprc_252d')

# Testing
for stock, stock_data in testing_stock_df.items():
    prc_highprc_252d = get_prc_highprc_252d(stock)

    # Merge the data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], prc_highprc_252d, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    testing_feature_df[stock][f'{stock}_prc_highprc_252d'] = testing_feature_df[stock][f'{stock}_prc_highprc_252d'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{stock}_prc_highprc_252d']):
        print(f"Feature f'{stock}_prc_highprc_252d' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}_prc_highprc_252d')

In [59]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

        date  AAPL_exponential_moving_average_5  AAPL_exponential_moving_average_10  AAPL_exponential_moving_average_21  AAPL_downside_deviation_5  AAPL_downside_deviation_10  AAPL_downside_deviation_21  AAPL_sortino_ratio_5  AAPL_sortino_ratio_10  AAPL_sortino_ratio_21  AAPL_mom12m  AAPL_mom1m  AAPL_mom6m  AAPL_mvel1  AAPL_rvol_21d  AAPL_chcsho_1m  AAPL_chcsho_3m  AAPL_chcsho_6m  AAPL_ami_126d  AAPL_age  AAPL_chmom  AAPL_maxret   AAPL_dolvol  AAPL_beta_60m  AAPL_zero_trades_1m  AAPL_zero_trades_6m  AAPL_zero_trades_12m  AAPL_betabab_1260d  AAPL_earnings_variability  AAPL_ocfq_saleq_std  AAPL_turnover_126d  AAPL_prc_highprc_252d
0 2016-07-01                          23.972500                           23.972500                           23.972500                   0.184606                    0.193321                    0.200102              0.006554               0.006258               0.006046     0.347234    0.026148    0.169584   23.804209       0.016383       -0.000876       -0.016

#### Feature 30: Residual momentum t-6 to t-1 (resff3_6_1)

In [60]:
def get_resff3_6_1(stock):
    global_factor = pd.read_excel(f"{DATA_DIR}/global_factor/{stock}_global_factor.xlsx")
    target_cols = ['Day of last price observation (date)', 'Residual momentum t-6 to t-1 (resff3_6_1)']
    new_cols = global_factor[target_cols].copy()

    # Rename the columns
    new_cols.rename(columns={target_cols[1]: f'{stock}_resff3_6_1'}, inplace=True)

    # Parse the date column
    new_cols[target_cols[0]] = parse_dates(new_cols[target_cols[0]])

    return new_cols

# Training
for stock, stock_data in training_stock_df.items():
    new_cols = get_resff3_6_1(stock)
    substring = '_resff3_6_1'
    date_substring = 'Day of last price observation (date)'

    # Merge the data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    # Backfill the missing values
    training_feature_df[stock][f'{stock}{substring}'] = training_feature_df[stock][f'{stock}{substring}'].bfill()

    if check_weird_data(training_feature_df[stock][f'{stock}{substring}']):
        print(f"Feature f'{stock}{substring}' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}{substring}')

# Testing
for stock, stock_data in testing_stock_df.items():
    new_cols = get_resff3_6_1(stock)
    substring = '_resff3_6_1'
    date_substring = 'Day of last price observation (date)'

    # Merge the data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    # Backfill the missing values
    testing_feature_df[stock][f'{stock}{substring}'] = testing_feature_df[stock][f'{stock}{substring}'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{stock}{substring}']):
        print(f"Feature f'{stock}{substring}' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}{substring}')


In [61]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

        date  AAPL_exponential_moving_average_5  AAPL_exponential_moving_average_10  AAPL_exponential_moving_average_21  AAPL_downside_deviation_5  AAPL_downside_deviation_10  AAPL_downside_deviation_21  AAPL_sortino_ratio_5  AAPL_sortino_ratio_10  AAPL_sortino_ratio_21  AAPL_mom12m  AAPL_mom1m  AAPL_mom6m  AAPL_mvel1  AAPL_rvol_21d  AAPL_chcsho_1m  AAPL_chcsho_3m  AAPL_chcsho_6m  AAPL_ami_126d  AAPL_age  AAPL_chmom  AAPL_maxret   AAPL_dolvol  AAPL_beta_60m  AAPL_zero_trades_1m  AAPL_zero_trades_6m  AAPL_zero_trades_12m  AAPL_betabab_1260d  AAPL_earnings_variability  AAPL_ocfq_saleq_std  AAPL_turnover_126d  AAPL_prc_highprc_252d  AAPL_resff3_6_1
0 2016-07-01                          23.972500                           23.972500                           23.972500                   0.184606                    0.193321                    0.200102              0.006554               0.006258               0.006046     0.347234    0.026148    0.169584   23.804209       0.016383       -0.00

#### Feature 31: 1 Year Non-Annual Seasonality (seas_1_1na)

In [62]:
def get_seas_1_1na(stock):
    global_factor = pd.read_excel(f"{DATA_DIR}/global_factor/{stock}_global_factor.xlsx")
    target_cols = ['Day of last price observation (date)', 'Year 1-lagged return, annual (seas_1_1an)']
    new_cols = global_factor[target_cols].copy()

    # Rename the columns
    new_cols.rename(columns={target_cols[1]: f'{stock}_seas_1_1na'}, inplace=True)

    # Parse the date column
    new_cols[target_cols[0]] = parse_dates(new_cols[target_cols[0]])

    return new_cols

# Training
for stock, stock_data in training_stock_df.items():
    new_cols = get_seas_1_1na(stock)
    substring = '_seas_1_1na'
    date_substring = 'Day of last price observation (date)'

    # Merge the data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    # Backfill the missing values
    training_feature_df[stock][f'{stock}{substring}'] = training_feature_df[stock][f'{stock}{substring}'].bfill()

    if check_weird_data(training_feature_df[stock][f'{stock}{substring}']):
        print(f"Feature f'{stock}{substring}' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}{substring}')

# Testing
for stock, stock_data in testing_stock_df.items():
    new_cols = get_seas_1_1na(stock)
    substring = '_seas_1_1na'
    date_substring = 'Day of last price observation (date)'

    # Merge the data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    # Backfill the missing values
    testing_feature_df[stock][f'{stock}{substring}'] = testing_feature_df[stock][f'{stock}{substring}'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{stock}{substring}']):
        print(f"Feature f'{stock}{substring}' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}{substring}')


In [63]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

        date  AAPL_exponential_moving_average_5  AAPL_exponential_moving_average_10  AAPL_exponential_moving_average_21  AAPL_downside_deviation_5  AAPL_downside_deviation_10  AAPL_downside_deviation_21  AAPL_sortino_ratio_5  AAPL_sortino_ratio_10  AAPL_sortino_ratio_21  AAPL_mom12m  AAPL_mom1m  AAPL_mom6m  AAPL_mvel1  AAPL_rvol_21d  AAPL_chcsho_1m  AAPL_chcsho_3m  AAPL_chcsho_6m  AAPL_ami_126d  AAPL_age  AAPL_chmom  AAPL_maxret   AAPL_dolvol  AAPL_beta_60m  AAPL_zero_trades_1m  AAPL_zero_trades_6m  AAPL_zero_trades_12m  AAPL_betabab_1260d  AAPL_earnings_variability  AAPL_ocfq_saleq_std  AAPL_turnover_126d  AAPL_prc_highprc_252d  AAPL_resff3_6_1  AAPL_seas_1_1na
0 2016-07-01                          23.972500                           23.972500                           23.972500                   0.184606                    0.193321                    0.200102              0.006554               0.006258               0.006046     0.347234    0.026148    0.169584   23.804209       0.0

#### Feature 32: Capital turnover (at_turnover)

In [64]:
def get_at_turnover(stock):
    global_factor = pd.read_excel(f"{DATA_DIR}/global_factor/{stock}_global_factor.xlsx")
    target_cols = ['Day of last price observation (date)', 'Capital turnover (at_turnover)']
    new_cols = global_factor[target_cols].copy()

    # Rename the columns
    new_cols.rename(columns={target_cols[1]: f'{stock}_at_turnover'}, inplace=True)

    # Parse the date column
    new_cols[target_cols[0]] = parse_dates(new_cols[target_cols[0]])

    return new_cols

# Training
for stock, stock_data in training_stock_df.items():
    new_cols = get_at_turnover(stock)
    substring = '_at_turnover'
    date_substring = 'Day of last price observation (date)'

    # Merge the data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    # Backfill the missing values
    training_feature_df[stock][f'{stock}{substring}'] = training_feature_df[stock][f'{stock}{substring}'].bfill()

    if check_weird_data(training_feature_df[stock][f'{stock}{substring}']):
        print(f"Feature f'{stock}{substring}' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}{substring}')

# Testing
for stock, stock_data in testing_stock_df.items():
    new_cols = get_at_turnover(stock)
    substring = '_at_turnover'
    date_substring = 'Day of last price observation (date)'

    # Merge the data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    # Backfill the missing values
    testing_feature_df[stock][f'{stock}{substring}'] = testing_feature_df[stock][f'{stock}{substring}'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{stock}{substring}']):
        print(f"Feature f'{stock}{substring}' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}{substring}')

In [65]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

        date  AAPL_exponential_moving_average_5  AAPL_exponential_moving_average_10  AAPL_exponential_moving_average_21  AAPL_downside_deviation_5  AAPL_downside_deviation_10  AAPL_downside_deviation_21  AAPL_sortino_ratio_5  AAPL_sortino_ratio_10  AAPL_sortino_ratio_21  AAPL_mom12m  AAPL_mom1m  AAPL_mom6m  AAPL_mvel1  AAPL_rvol_21d  AAPL_chcsho_1m  AAPL_chcsho_3m  AAPL_chcsho_6m  AAPL_ami_126d  AAPL_age  AAPL_chmom  AAPL_maxret   AAPL_dolvol  AAPL_beta_60m  AAPL_zero_trades_1m  AAPL_zero_trades_6m  AAPL_zero_trades_12m  AAPL_betabab_1260d  AAPL_earnings_variability  AAPL_ocfq_saleq_std  AAPL_turnover_126d  AAPL_prc_highprc_252d  AAPL_resff3_6_1  AAPL_seas_1_1na  AAPL_at_turnover
0 2016-07-01                          23.972500                           23.972500                           23.972500                   0.184606                    0.193321                    0.200102              0.006554               0.006258               0.006046     0.347234    0.026148    0.169584   2

#### Feature 33: US 30 Day Bill Returns (T-30)

In [66]:
def get_T_30():
    global_factor = pd.read_excel(f"{DATA_DIR}/Monetary and Market Condition/US 30 Day Bill Returns.xlsx")
    target_cols = ['Calendar Date', '30 Day Bill Returns']
    new_cols = global_factor[target_cols].copy()

    # Rename the columns
    new_cols.rename(columns={target_cols[1]: f'T_30'}, inplace=True)

    # Parse the date column
    new_cols[target_cols[0]] = parse_dates(new_cols[target_cols[0]])

    return new_cols

# Training
for stock, stock_data in training_stock_df.items():
    new_cols = get_T_30()
    substring = 'T_30'
    date_substring = 'Calendar Date'

    # Merge the data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    # Backfill the missing values
    training_feature_df[stock][f'{substring}'] = training_feature_df[stock][f'{substring}'].bfill()

    if check_weird_data(training_feature_df[stock][f'{substring}']):
        print(f"Feature f'{substring}' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{substring}')

# Testing
for stock, stock_data in testing_stock_df.items():
    new_cols = get_T_30()
    substring = 'T_30'
    date_substring = 'Calendar Date'

    # Merge the data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    # Backfill the missing values
    testing_feature_df[stock][f'{substring}'] = testing_feature_df[stock][f'{substring}'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{substring}']):
        print(f"Feature f'{substring}' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{substring}')


In [67]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

        date  AAPL_exponential_moving_average_5  AAPL_exponential_moving_average_10  AAPL_exponential_moving_average_21  AAPL_downside_deviation_5  AAPL_downside_deviation_10  AAPL_downside_deviation_21  AAPL_sortino_ratio_5  AAPL_sortino_ratio_10  AAPL_sortino_ratio_21  AAPL_mom12m  AAPL_mom1m  AAPL_mom6m  AAPL_mvel1  AAPL_rvol_21d  AAPL_chcsho_1m  AAPL_chcsho_3m  AAPL_chcsho_6m  AAPL_ami_126d  AAPL_age  AAPL_chmom  AAPL_maxret   AAPL_dolvol  AAPL_beta_60m  AAPL_zero_trades_1m  AAPL_zero_trades_6m  AAPL_zero_trades_12m  AAPL_betabab_1260d  AAPL_earnings_variability  AAPL_ocfq_saleq_std  AAPL_turnover_126d  AAPL_prc_highprc_252d  AAPL_resff3_6_1  AAPL_seas_1_1na  AAPL_at_turnover      T_30
0 2016-07-01                          23.972500                           23.972500                           23.972500                   0.184606                    0.193321                    0.200102              0.006554               0.006258               0.006046     0.347234    0.026148    0.

#### Feature 34: CBOE NASDAQ Volatility Index (Log-Difference and EWMA with halflife 63 days) (VXN_LogDiff_EWMAhl63) (Self-Constructed)

In [68]:
def get_VXN_LogDiff_EWMAhl63(stock):
    vix = pd.read_excel(f"{DATA_DIR}/Monetary and Market Condition/CBOE NASDAQ Volatility Index - Close.xlsx")
    target_cols = ['Date', 'CBOE NASDAQ Volatility Index - Close']
    new_cols = vix[target_cols].copy()

    # Compute the log-difference of the VIX index values
    new_cols[target_cols[1]] = np.log(new_cols[target_cols[1]]) - np.log(new_cols[target_cols[1]].shift(1))

    # Apply EWMA smoothing with a halflife of 63 days
    new_cols[target_cols[1]] = new_cols[target_cols[1]].ewm(halflife=63, adjust=False).mean()

    # Rename the columns
    new_cols.rename(columns={target_cols[1]: f'{stock}_VXN_LogDiff_EWMAhl63'}, inplace=True)

    # Parse the date column
    new_cols[target_cols[0]] = parse_dates(new_cols[target_cols[0]])

    return new_cols

# Training
for stock, stock_data in training_stock_df.items():
    new_cols = get_VXN_LogDiff_EWMAhl63(stock)
    substring = '_VXN_LogDiff_EWMAhl63'
    date_substring = 'Date'

    # Merge the data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    if check_weird_data(training_feature_df[stock][f'{stock}{substring}']):
        print(f"Feature f'{stock}{substring}' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}{substring}')

# Testing
for stock, stock_data in testing_stock_df.items():
    new_cols = get_VXN_LogDiff_EWMAhl63(stock)
    substring = '_VXN_LogDiff_EWMAhl63'
    date_substring = 'Date'

    # Merge the data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    if check_weird_data(testing_feature_df[stock][f'{stock}{substring}']):
        print(f"Feature f'{stock}{substring}' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}{substring}')

Feature f'META_VXN_LogDiff_EWMAhl63' contains weird data in training data


In [69]:
# Training
for stock, weird_columns in training_weird_columns_dict.items():
    if weird_columns:
        training_feature_df[stock] = handle_weird_data_with_knn_imputer(training_feature_df[stock], weird_columns)


In [70]:
# Check the data again
for stock, stock_data in training_feature_df.items():
    if check_weird_data(stock_data):
        print(f"The {stock} stock price data still contains weird data in training data")

In [71]:
# Clear weird_columns_dict, only clear the value, not the key
for stock in stock_list:
    training_weird_columns_dict[stock] = []

In [72]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

        date  AAPL_exponential_moving_average_5  AAPL_exponential_moving_average_10  AAPL_exponential_moving_average_21  AAPL_downside_deviation_5  AAPL_downside_deviation_10  AAPL_downside_deviation_21  AAPL_sortino_ratio_5  AAPL_sortino_ratio_10  AAPL_sortino_ratio_21  AAPL_mom12m  AAPL_mom1m  AAPL_mom6m  AAPL_mvel1  AAPL_rvol_21d  AAPL_chcsho_1m  AAPL_chcsho_3m  AAPL_chcsho_6m  AAPL_ami_126d  AAPL_age  AAPL_chmom  AAPL_maxret   AAPL_dolvol  AAPL_beta_60m  AAPL_zero_trades_1m  AAPL_zero_trades_6m  AAPL_zero_trades_12m  AAPL_betabab_1260d  AAPL_earnings_variability  AAPL_ocfq_saleq_std  AAPL_turnover_126d  AAPL_prc_highprc_252d  AAPL_resff3_6_1  AAPL_seas_1_1na  AAPL_at_turnover      T_30  AAPL_VXN_LogDiff_EWMAhl63
0 2016-07-01                          23.972500                           23.972500                           23.972500                   0.184606                    0.193321                    0.200102              0.006554               0.006258               0.006046    

#### Feature 35: Rate of Change in Consumer Price Index (CPI_ROC) 

In [73]:
def get_CPI_ROC():
    CPI_ROC = pd.read_excel(f"{DATA_DIR}/Monetary and Market Condition/US Rate of Change in Consumer Price Index.xlsx")
    target_cols = ['Calendar Date', 'Rate of Change in Consumer Price Index']
    new_cols = CPI_ROC[target_cols].copy()

    # Rename the columns
    new_cols.rename(columns={target_cols[1]: f'CPI_ROC'}, inplace=True)

    # Parse the date column
    new_cols[target_cols[0]] = parse_dates(new_cols[target_cols[0]])

    return new_cols


# Training
for stock, stock_data in training_stock_df.items():
    new_cols = get_CPI_ROC()
    substring = 'CPI_ROC'
    date_substring = 'Calendar Date'

    # Merge the data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    # Backfill the missing values
    training_feature_df[stock][f'{substring}'] = training_feature_df[stock][f'{substring}'].bfill()

    if check_weird_data(training_feature_df[stock][f'{substring}']):
        print(f"Feature f'{substring}' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{substring}')

# Testing
for stock, stock_data in testing_stock_df.items():
    new_cols = get_CPI_ROC()
    substring = 'CPI_ROC'
    date_substring = 'Calendar Date'

    # Merge the data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)
    
    # Backfill the missing values
    testing_feature_df[stock][f'{substring}'] = testing_feature_df[stock][f'{substring}'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{substring}']):
        print(f"Feature f'{substring}' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{substring}')

In [74]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

        date  AAPL_exponential_moving_average_5  AAPL_exponential_moving_average_10  AAPL_exponential_moving_average_21  AAPL_downside_deviation_5  AAPL_downside_deviation_10  AAPL_downside_deviation_21  AAPL_sortino_ratio_5  AAPL_sortino_ratio_10  AAPL_sortino_ratio_21  AAPL_mom12m  AAPL_mom1m  AAPL_mom6m  AAPL_mvel1  AAPL_rvol_21d  AAPL_chcsho_1m  AAPL_chcsho_3m  AAPL_chcsho_6m  AAPL_ami_126d  AAPL_age  AAPL_chmom  AAPL_maxret   AAPL_dolvol  AAPL_beta_60m  AAPL_zero_trades_1m  AAPL_zero_trades_6m  AAPL_zero_trades_12m  AAPL_betabab_1260d  AAPL_earnings_variability  AAPL_ocfq_saleq_std  AAPL_turnover_126d  AAPL_prc_highprc_252d  AAPL_resff3_6_1  AAPL_seas_1_1na  AAPL_at_turnover      T_30  AAPL_VXN_LogDiff_EWMAhl63   CPI_ROC
0 2016-07-01                          23.972500                           23.972500                           23.972500                   0.184606                    0.193321                    0.200102              0.006554               0.006258               0.

#### Feature 36: Yield curve slope (10-year minus 2-year) with EWMA halflives 10 days  (T10Y2Y_EWMAhl10)  (Self-Constructed)

In [75]:
def get_T10Y2Y_EWMAhl10():
    T10Y2Y = pd.read_excel(f"{DATA_DIR}/Monetary and Market Condition/T10Y2Y.xlsx")
    target_cols = ['observation_date', 'T10Y2Y']
    new_cols = T10Y2Y[target_cols].copy()

    new_cols['T10Y2Y'] = new_cols['T10Y2Y'].ewm(halflife=10, adjust=False).mean()

    # Rename the columns
    new_cols.rename(columns={target_cols[1]: f'T10Y2Y_EWMAhl10'}, inplace=True)

    # Parse the date column
    new_cols[target_cols[0]] = parse_dates(new_cols[target_cols[0]])

    return new_cols

# Training
for stock, stock_data in training_stock_df.items():
    new_cols = get_T10Y2Y_EWMAhl10()
    substring = 'T10Y2Y_EWMAhl10'
    date_substring = 'observation_date'

    # Merge the data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    if check_weird_data(training_feature_df[stock][f'{substring}']):
        print(f"Feature f'{substring}' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{substring}')


# Testing
for stock, stock_data in testing_stock_df.items():
    new_cols = get_T10Y2Y_EWMAhl10()
    substring = 'T10Y2Y_EWMAhl10'
    date_substring = 'observation_date'

    # Merge the data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    if check_weird_data(testing_feature_df[stock][f'{substring}']):
        print(f"Feature f'{substring}' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{substring}')
 

In [76]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

        date  AAPL_exponential_moving_average_5  AAPL_exponential_moving_average_10  AAPL_exponential_moving_average_21  AAPL_downside_deviation_5  AAPL_downside_deviation_10  AAPL_downside_deviation_21  AAPL_sortino_ratio_5  AAPL_sortino_ratio_10  AAPL_sortino_ratio_21  AAPL_mom12m  AAPL_mom1m  AAPL_mom6m  AAPL_mvel1  AAPL_rvol_21d  AAPL_chcsho_1m  AAPL_chcsho_3m  AAPL_chcsho_6m  AAPL_ami_126d  AAPL_age  AAPL_chmom  AAPL_maxret   AAPL_dolvol  AAPL_beta_60m  AAPL_zero_trades_1m  AAPL_zero_trades_6m  AAPL_zero_trades_12m  AAPL_betabab_1260d  AAPL_earnings_variability  AAPL_ocfq_saleq_std  AAPL_turnover_126d  AAPL_prc_highprc_252d  AAPL_resff3_6_1  AAPL_seas_1_1na  AAPL_at_turnover      T_30  AAPL_VXN_LogDiff_EWMAhl63   CPI_ROC  T10Y2Y_EWMAhl10
0 2016-07-01                          23.972500                           23.972500                           23.972500                   0.184606                    0.193321                    0.200102              0.006554               0.006258

#### Feature 37: Yield curve slope (10-year minus 2-year) with Log-Difference and EWMA with halflife 21 days  (T10Y2Y_LogDiff_EWMAhl21) (Self-Constructed)

In [77]:
def get_T10Y2Y_LogDiff_EWMAhl21():
    T10Y2Y = pd.read_excel(f"{DATA_DIR}/Monetary and Market Condition/T10Y2Y.xlsx")
    target_cols = ['observation_date', 'T10Y2Y']
    new_cols = T10Y2Y[target_cols].copy()

    # Add a small constant epsilon to avoid taking the log of zero.
    epsilon = 1e-7
    with np.errstate(divide='ignore', invalid='ignore'):
        # Compute the log-difference safely.
        new_cols[target_cols[1]] = np.log(new_cols[target_cols[1]] + epsilon) - \
                                    np.log(new_cols[target_cols[1]].shift(1) + epsilon)

    # Apply EWMA smoothing with a halflife of 21 days.
    new_cols[target_cols[1]] = new_cols[target_cols[1]].ewm(halflife=21, adjust=False).mean()

    # Rename the column.
    new_cols.rename(columns={target_cols[1]: 'T10Y2Y_LogDiff_EWMAhl21'}, inplace=True)

    # Parse the date column.
    new_cols[target_cols[0]] = parse_dates(new_cols[target_cols[0]])

    return new_cols

# Training
for stock, stock_data in training_stock_df.items():
    new_cols = get_T10Y2Y_LogDiff_EWMAhl21()
    substring = 'T10Y2Y_LogDiff_EWMAhl21'
    date_substring = 'observation_date'

    # Merge the data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    if check_weird_data(training_feature_df[stock][f'{substring}']):
        print(f"Feature f'{substring}' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{substring}')

# Testing
for stock, stock_data in testing_stock_df.items():
    new_cols = get_T10Y2Y_LogDiff_EWMAhl21()
    substring = 'T10Y2Y_LogDiff_EWMAhl21'
    date_substring = 'observation_date'

    # Merge the data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    if check_weird_data(testing_feature_df[stock][f'{substring}']):
        print(f"Feature f'{substring}' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{substring}')


Feature f'T10Y2Y_LogDiff_EWMAhl21' contains weird data in training data


In [78]:
for stock, weird_columns in training_weird_columns_dict.items():
    if weird_columns:
        training_feature_df[stock] = handle_weird_data_with_knn_imputer(training_feature_df[stock], weird_columns)

# Check the data again
for stock, stock_data in training_feature_df.items():
    if check_weird_data(stock_data):
        print(f"The {stock} stock price data still contains weird data in training data")

In [79]:
# Clear weird_columns_dict, only clear the value, not the key
for stock in stock_list:
    training_weird_columns_dict[stock] = []

In [80]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

        date  AAPL_exponential_moving_average_5  AAPL_exponential_moving_average_10  AAPL_exponential_moving_average_21  AAPL_downside_deviation_5  AAPL_downside_deviation_10  AAPL_downside_deviation_21  AAPL_sortino_ratio_5  AAPL_sortino_ratio_10  AAPL_sortino_ratio_21  AAPL_mom12m  AAPL_mom1m  AAPL_mom6m  AAPL_mvel1  AAPL_rvol_21d  AAPL_chcsho_1m  AAPL_chcsho_3m  AAPL_chcsho_6m  AAPL_ami_126d  AAPL_age  AAPL_chmom  AAPL_maxret   AAPL_dolvol  AAPL_beta_60m  AAPL_zero_trades_1m  AAPL_zero_trades_6m  AAPL_zero_trades_12m  AAPL_betabab_1260d  AAPL_earnings_variability  AAPL_ocfq_saleq_std  AAPL_turnover_126d  AAPL_prc_highprc_252d  AAPL_resff3_6_1  AAPL_seas_1_1na  AAPL_at_turnover      T_30  AAPL_VXN_LogDiff_EWMAhl63   CPI_ROC  T10Y2Y_EWMAhl10  T10Y2Y_LogDiff_EWMAhl21
0 2016-07-01                          23.972500                           23.972500                           23.972500                   0.184606                    0.193321                    0.200102              0.0065

#### Feature 38: US treasury 2-year yield with Log-Difference and EWMA with halflife 21 days (DGS2_LogDiff_EWMAhl21) (Self-Constructed)

In [81]:
def get_DGS2_LogDiff_EWMAhl21():
    DGS2 = pd.read_excel(f"{DATA_DIR}/Monetary and Market Condition/DGS2.xlsx")
    target_cols = ['observation_date', 'DGS2']
    new_cols = DGS2[target_cols].copy()

    # Add a small constant epsilon to avoid taking the log of zero.
    epsilon = 1e-7
    with np.errstate(divide='ignore', invalid='ignore'):
        # Compute the log-difference safely.
        new_cols[target_cols[1]] = np.log(new_cols[target_cols[1]] + epsilon) - \
                                    np.log(new_cols[target_cols[1]].shift(1) + epsilon)

    # Apply EWMA smoothing with a halflife of 21 days.
    new_cols[target_cols[1]] = new_cols[target_cols[1]].ewm(halflife=21, adjust=False).mean()

    # Rename the column.
    new_cols.rename(columns={target_cols[1]: 'DGS2_LogDiff_EWMAhl21'}, inplace=True)

    # Parse the date column.
    new_cols[target_cols[0]] = parse_dates(new_cols[target_cols[0]])

    return new_cols

# Training
for stock, stock_data in training_stock_df.items():
    new_cols = get_DGS2_LogDiff_EWMAhl21()
    substring = 'DGS2_LogDiff_EWMAhl21'
    date_substring = 'observation_date'

    # Merge the data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    if check_weird_data(training_feature_df[stock][f'{substring}']):
        print(f"Feature f'{substring}' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{substring}')

# Testing
for stock, stock_data in testing_stock_df.items():
    new_cols = get_DGS2_LogDiff_EWMAhl21()
    substring = 'DGS2_LogDiff_EWMAhl21'
    date_substring = 'observation_date'

    # Merge the data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    if check_weird_data(testing_feature_df[stock][f'{substring}']):
        print(f"Feature f'{substring}' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{substring}')

Feature f'DGS2_LogDiff_EWMAhl21' contains weird data in training data


In [82]:
# Hanld weird data in training data
for stock, weird_columns in training_weird_columns_dict.items():
    if weird_columns:
        training_feature_df[stock] = handle_weird_data_with_knn_imputer(training_feature_df[stock], weird_columns)

# Check the data again
for stock, stock_data in training_feature_df.items():
    if check_weird_data(stock_data):
        print(f"The {stock} stock price data still contains weird data in training data")

In [83]:
# Clear weird_columns_dict, only clear the value, not the key
for stock in stock_list:
    training_weird_columns_dict[stock] = []
    testing_weird_columns_dict[stock] = []

In [84]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

        date  AAPL_exponential_moving_average_5  AAPL_exponential_moving_average_10  AAPL_exponential_moving_average_21  AAPL_downside_deviation_5  AAPL_downside_deviation_10  AAPL_downside_deviation_21  AAPL_sortino_ratio_5  AAPL_sortino_ratio_10  AAPL_sortino_ratio_21  AAPL_mom12m  AAPL_mom1m  AAPL_mom6m  AAPL_mvel1  AAPL_rvol_21d  AAPL_chcsho_1m  AAPL_chcsho_3m  AAPL_chcsho_6m  AAPL_ami_126d  AAPL_age  AAPL_chmom  AAPL_maxret   AAPL_dolvol  AAPL_beta_60m  AAPL_zero_trades_1m  AAPL_zero_trades_6m  AAPL_zero_trades_12m  AAPL_betabab_1260d  AAPL_earnings_variability  AAPL_ocfq_saleq_std  AAPL_turnover_126d  AAPL_prc_highprc_252d  AAPL_resff3_6_1  AAPL_seas_1_1na  AAPL_at_turnover      T_30  AAPL_VXN_LogDiff_EWMAhl63   CPI_ROC  T10Y2Y_EWMAhl10  T10Y2Y_LogDiff_EWMAhl21  DGS2_LogDiff_EWMAhl21
0 2016-07-01                          23.972500                           23.972500                           23.972500                   0.184606                    0.193321                    0.200

#### Feature 39: Real M2 money supply growth (12-month rate of change) (M2_RealGrowth) (Self-Constructed)

In [85]:
def get_M2_RealGrowth():
    # Load the M2 and CPI data
    M2 = pd.read_excel(f"{DATA_DIR}/Monetary and Market Condition/M2SL.xlsx")
    CPI = pd.read_excel(f"{DATA_DIR}/Monetary and Market Condition/CPIAUCSL.xlsx")

    # Select relevant columns
    M2_cols = ['observation_date', 'M2SL']
    CPI_cols = ['observation_date', 'CPIAUCSL']
    M2_data = M2[M2_cols].copy()
    CPI_data = CPI[CPI_cols].copy()

    # Merge M2 and CPI data on observation_date
    merged_data = pd.merge(M2_data, CPI_data, on='observation_date', how='inner')

    # Calculate real M2 growth (12-month rate of change)
    merged_data['M2_RealGrowth'] = (merged_data['M2SL'] / merged_data['CPIAUCSL']) - \
                                   (merged_data['M2SL'].shift(12) / merged_data['CPIAUCSL'].shift(12))
    
    # Parse the date column
    merged_data['observation_date'] = pd.to_datetime(merged_data['observation_date'])

    return merged_data[['observation_date', 'M2_RealGrowth']]


# Training
for stock, stock_data in training_stock_df.items():
    new_cols = get_M2_RealGrowth()
    substring = 'M2_RealGrowth'
    date_substring = 'observation_date'

    # Merge the data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    # Backfill the missing values
    training_feature_df[stock][f'{substring}'] = training_feature_df[stock][f'{substring}'].bfill()

    if check_weird_data(training_feature_df[stock][f'{substring}']):
        print(f"Feature f'{substring}' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{substring}')

# Testing
for stock, stock_data in testing_stock_df.items():
    new_cols = get_M2_RealGrowth()
    substring = 'M2_RealGrowth'
    date_substring = 'observation_date'

    # Merge the data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    # Backfill the missing values
    testing_feature_df[stock][f'{substring}'] = testing_feature_df[stock][f'{substring}'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{substring}']):
        print(f"Feature f'{substring}' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{substring}')


Feature f'M2_RealGrowth' contains weird data in training data
Feature f'M2_RealGrowth' contains weird data in training data
Feature f'M2_RealGrowth' contains weird data in training data
Feature f'M2_RealGrowth' contains weird data in training data
Feature f'M2_RealGrowth' contains weird data in training data
Feature f'M2_RealGrowth' contains weird data in training data
Feature f'M2_RealGrowth' contains weird data in training data
Feature f'M2_RealGrowth' contains weird data in training data
Feature f'M2_RealGrowth' contains weird data in training data
Feature f'M2_RealGrowth' contains weird data in testing data
Feature f'M2_RealGrowth' contains weird data in testing data
Feature f'M2_RealGrowth' contains weird data in testing data
Feature f'M2_RealGrowth' contains weird data in testing data
Feature f'M2_RealGrowth' contains weird data in testing data
Feature f'M2_RealGrowth' contains weird data in testing data
Feature f'M2_RealGrowth' contains weird data in testing data
Feature f'M2_Re

In [86]:
# Training
for stock, weird_columns in training_weird_columns_dict.items():
    if weird_columns:
        training_feature_df[stock] = handle_weird_data_with_knn_imputer(training_feature_df[stock], weird_columns)

# Testing
for stock, weird_columns in testing_weird_columns_dict.items():
    if weird_columns:
        testing_feature_df[stock] = handle_weird_data_with_knn_imputer(testing_feature_df[stock], weird_columns)

# Check the data again
for stock, stock_data in training_feature_df.items():
    if check_weird_data(stock_data):
        print(f"The {stock} stock price data still contains weird data in training data")

for stock, stock_data in testing_feature_df.items():
    if check_weird_data(stock_data):
        print(f"The {stock} stock price data still contains weird data in testing data")


In [87]:
# Clear weird_columns_dict, only clear the value, not the key
for stock in stock_list:
    training_weird_columns_dict[stock] = []
    testing_weird_columns_dict[stock] = []

In [88]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

        date  AAPL_exponential_moving_average_5  AAPL_exponential_moving_average_10  AAPL_exponential_moving_average_21  AAPL_downside_deviation_5  AAPL_downside_deviation_10  AAPL_downside_deviation_21  AAPL_sortino_ratio_5  AAPL_sortino_ratio_10  AAPL_sortino_ratio_21  AAPL_mom12m  AAPL_mom1m  AAPL_mom6m  AAPL_mvel1  AAPL_rvol_21d  AAPL_chcsho_1m  AAPL_chcsho_3m  AAPL_chcsho_6m  AAPL_ami_126d  AAPL_age  AAPL_chmom  AAPL_maxret   AAPL_dolvol  AAPL_beta_60m  AAPL_zero_trades_1m  AAPL_zero_trades_6m  AAPL_zero_trades_12m  AAPL_betabab_1260d  AAPL_earnings_variability  AAPL_ocfq_saleq_std  AAPL_turnover_126d  AAPL_prc_highprc_252d  AAPL_resff3_6_1  AAPL_seas_1_1na  AAPL_at_turnover      T_30  AAPL_VXN_LogDiff_EWMAhl63   CPI_ROC  T10Y2Y_EWMAhl10  T10Y2Y_LogDiff_EWMAhl21  DGS2_LogDiff_EWMAhl21  M2_RealGrowth
0 2016-07-01                          23.972500                           23.972500                           23.972500                   0.184606                    0.193321          

#### Feature 40: Financial Conditions Index with Log-Difference and EWMA with halflife 21 days (NFCI_LogDiff_EWMAhl21) (Self-Constructed)

In [89]:
def get_NFCI_LogDiff_EWMAhl21():
    NFCI = pd.read_excel(f"{DATA_DIR}/Monetary and Market Condition/NFCI.xlsx")
    target_cols = ['observation_date', 'NFCI']
    new_cols = NFCI[target_cols].copy()

    # Add a small constant epsilon to avoid taking the log of zero.
    epsilon = 1e-7
    with np.errstate(divide='ignore', invalid='ignore'):
        # Compute the log-difference safely.
        new_cols[target_cols[1]] = np.log(new_cols[target_cols[1]] + epsilon) - \
                                    np.log(new_cols[target_cols[1]].shift(1) + epsilon)
        
    # Apply EWMA smoothing with a halflife of 21 days.
    new_cols[target_cols[1]] = new_cols[target_cols[1]].ewm(halflife=21, adjust=False).mean()

    # Rename the column.
    new_cols.rename(columns={target_cols[1]: 'NFCI_LogDiff_EWMAhl21'}, inplace=True)

    # Parse the date column.
    new_cols[target_cols[0]] = parse_dates(new_cols[target_cols[0]])

    return new_cols
    
# Training
for stock, stock_data in training_stock_df.items():
    new_cols = get_NFCI_LogDiff_EWMAhl21()
    substring = 'NFCI_LogDiff_EWMAhl21'
    date_substring = 'observation_date'

    # Merge the data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    if check_weird_data(training_feature_df[stock][f'{substring}']):
        print(f"Feature f'{substring}' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{substring}')

# Testing
for stock, stock_data in testing_stock_df.items():
    new_cols = get_NFCI_LogDiff_EWMAhl21()
    substring = 'NFCI_LogDiff_EWMAhl21'
    date_substring = 'observation_date'

    # Merge the data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    if check_weird_data(testing_feature_df[stock][f'{substring}']):
        print(f"Feature f'{substring}' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{substring}')

Feature f'NFCI_LogDiff_EWMAhl21' contains weird data in training data
Feature f'NFCI_LogDiff_EWMAhl21' contains weird data in training data
Feature f'NFCI_LogDiff_EWMAhl21' contains weird data in training data
Feature f'NFCI_LogDiff_EWMAhl21' contains weird data in training data
Feature f'NFCI_LogDiff_EWMAhl21' contains weird data in training data
Feature f'NFCI_LogDiff_EWMAhl21' contains weird data in training data
Feature f'NFCI_LogDiff_EWMAhl21' contains weird data in training data
Feature f'NFCI_LogDiff_EWMAhl21' contains weird data in training data
Feature f'NFCI_LogDiff_EWMAhl21' contains weird data in training data
Feature f'NFCI_LogDiff_EWMAhl21' contains weird data in testing data
Feature f'NFCI_LogDiff_EWMAhl21' contains weird data in testing data
Feature f'NFCI_LogDiff_EWMAhl21' contains weird data in testing data
Feature f'NFCI_LogDiff_EWMAhl21' contains weird data in testing data
Feature f'NFCI_LogDiff_EWMAhl21' contains weird data in testing data
Feature f'NFCI_LogDiff_EW

In [90]:
# Training
for stock, weird_columns in training_weird_columns_dict.items():
    if weird_columns:
        training_feature_df[stock] = handle_weird_data_with_knn_imputer(training_feature_df[stock], weird_columns)

# Testing
for stock, weird_columns in testing_weird_columns_dict.items():
    if weird_columns:
        testing_feature_df[stock] = handle_weird_data_with_knn_imputer(testing_feature_df[stock], weird_columns)

# Check the data again
for stock, stock_data in training_feature_df.items():
    if check_weird_data(stock_data):
        print(f"The {stock} stock price data still contains weird data in training data")

for stock, stock_data in testing_feature_df.items():
    if check_weird_data(stock_data):
        print(f"The {stock} stock price data still contains weird data in testing data")


In [91]:
# Clear weird_columns_dict, only clear the value, not the key
for stock in stock_list:
    training_weird_columns_dict[stock] = []
    testing_weird_columns_dict[stock] = []

In [92]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

        date  AAPL_exponential_moving_average_5  AAPL_exponential_moving_average_10  AAPL_exponential_moving_average_21  AAPL_downside_deviation_5  AAPL_downside_deviation_10  AAPL_downside_deviation_21  AAPL_sortino_ratio_5  AAPL_sortino_ratio_10  AAPL_sortino_ratio_21  AAPL_mom12m  AAPL_mom1m  AAPL_mom6m  AAPL_mvel1  AAPL_rvol_21d  AAPL_chcsho_1m  AAPL_chcsho_3m  AAPL_chcsho_6m  AAPL_ami_126d  AAPL_age  AAPL_chmom  AAPL_maxret   AAPL_dolvol  AAPL_beta_60m  AAPL_zero_trades_1m  AAPL_zero_trades_6m  AAPL_zero_trades_12m  AAPL_betabab_1260d  AAPL_earnings_variability  AAPL_ocfq_saleq_std  AAPL_turnover_126d  AAPL_prc_highprc_252d  AAPL_resff3_6_1  AAPL_seas_1_1na  AAPL_at_turnover      T_30  AAPL_VXN_LogDiff_EWMAhl63   CPI_ROC  T10Y2Y_EWMAhl10  T10Y2Y_LogDiff_EWMAhl21  DGS2_LogDiff_EWMAhl21  M2_RealGrowth  NFCI_LogDiff_EWMAhl21
0 2016-07-01                          23.972500                           23.972500                           23.972500                   0.184606               

#### Feature 41: Economic Policy Uncertainty Index smoothed with 21-day EWMA (USEPUINDXD_EWMAhl21_t) (Self-Constructed)

In [93]:
def get_USEPUINDXD_EWMAhl21():
    USEPUINDXD = pd.read_excel(f"{DATA_DIR}/Monetary and Market Condition/USEPUINDXD.xlsx")
    target_cols = ['observation_date', 'USEPUINDXD']
    new_cols = USEPUINDXD[target_cols].copy()

    # Apply EWMA smoothing with a halflife of 21 days.
    new_cols[target_cols[1]] = new_cols[target_cols[1]].ewm(halflife=21, adjust=False).mean()

    # Rename the column.
    new_cols.rename(columns={target_cols[1]: 'USEPUINDXD_EWMAhl21'}, inplace=True)

    # Parse the date column.
    new_cols[target_cols[0]] = parse_dates(new_cols[target_cols[0]])

    return new_cols

# Training
for stock, stock_data in training_stock_df.items():
    new_cols = get_USEPUINDXD_EWMAhl21()
    substring = 'USEPUINDXD_EWMAhl21'
    date_substring = 'observation_date'

    # Merge the data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    if check_weird_data(training_feature_df[stock][f'{substring}']):
        print(f"Feature f'{substring}' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{substring}')


# Testing
for stock, stock_data in testing_stock_df.items():
    new_cols = get_USEPUINDXD_EWMAhl21()
    substring = 'USEPUINDXD_EWMAhl21'
    date_substring = 'observation_date'

    # Merge the data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    if check_weird_data(testing_feature_df[stock][f'{substring}']):
        print(f"Feature f'{substring}' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{substring}')

In [94]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

        date  AAPL_exponential_moving_average_5  AAPL_exponential_moving_average_10  AAPL_exponential_moving_average_21  AAPL_downside_deviation_5  AAPL_downside_deviation_10  AAPL_downside_deviation_21  AAPL_sortino_ratio_5  AAPL_sortino_ratio_10  AAPL_sortino_ratio_21  AAPL_mom12m  AAPL_mom1m  AAPL_mom6m  AAPL_mvel1  AAPL_rvol_21d  AAPL_chcsho_1m  AAPL_chcsho_3m  AAPL_chcsho_6m  AAPL_ami_126d  AAPL_age  AAPL_chmom  AAPL_maxret   AAPL_dolvol  AAPL_beta_60m  AAPL_zero_trades_1m  AAPL_zero_trades_6m  AAPL_zero_trades_12m  AAPL_betabab_1260d  AAPL_earnings_variability  AAPL_ocfq_saleq_std  AAPL_turnover_126d  AAPL_prc_highprc_252d  AAPL_resff3_6_1  AAPL_seas_1_1na  AAPL_at_turnover      T_30  AAPL_VXN_LogDiff_EWMAhl63   CPI_ROC  T10Y2Y_EWMAhl10  T10Y2Y_LogDiff_EWMAhl21  DGS2_LogDiff_EWMAhl21  M2_RealGrowth  NFCI_LogDiff_EWMAhl21  USEPUINDXD_EWMAhl21
0 2016-07-01                          23.972500                           23.972500                           23.972500                   0.

#### Feature 42: University of Michigan: Consumer Sentiment (UMCSENT)

In [95]:
def get_UMCSENT():
    UMCSENT = pd.read_excel(f"{DATA_DIR}/Monetary and Market Condition/UMCSENT.xlsx")
    target_cols = ['observation_date', 'UMCSENT']
    new_cols = UMCSENT[target_cols].copy()

    # Rename the columns
    new_cols.rename(columns={target_cols[1]: 'UMCSENT'}, inplace=True)

    # Parse the date column
    new_cols[target_cols[0]] = parse_dates(new_cols[target_cols[0]])

    return new_cols

# Training
for stock, stock_data in training_stock_df.items():
    new_cols = get_UMCSENT()
    substring = 'UMCSENT'
    date_substring = 'observation_date'

    # Merge the data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    # Backfill the missing values
    training_feature_df[stock][f'{substring}'] = training_feature_df[stock][f'{substring}'].bfill()

    if check_weird_data(training_feature_df[stock][f'{substring}']):
        print(f"Feature f'{substring}' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{substring}')

# Testing
for stock, stock_data in testing_stock_df.items():
    new_cols = get_UMCSENT()
    substring = 'UMCSENT'
    date_substring = 'observation_date'

    # Merge the data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    # Backfill the missing values
    testing_feature_df[stock][f'{substring}'] = testing_feature_df[stock][f'{substring}'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{substring}']):
        print(f"Feature f'{substring}' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{substring}')

Feature f'UMCSENT' contains weird data in training data
Feature f'UMCSENT' contains weird data in training data
Feature f'UMCSENT' contains weird data in training data
Feature f'UMCSENT' contains weird data in training data
Feature f'UMCSENT' contains weird data in training data
Feature f'UMCSENT' contains weird data in training data
Feature f'UMCSENT' contains weird data in training data
Feature f'UMCSENT' contains weird data in training data
Feature f'UMCSENT' contains weird data in training data
Feature f'UMCSENT' contains weird data in testing data
Feature f'UMCSENT' contains weird data in testing data
Feature f'UMCSENT' contains weird data in testing data
Feature f'UMCSENT' contains weird data in testing data
Feature f'UMCSENT' contains weird data in testing data
Feature f'UMCSENT' contains weird data in testing data
Feature f'UMCSENT' contains weird data in testing data
Feature f'UMCSENT' contains weird data in testing data
Feature f'UMCSENT' contains weird data in testing data


In [96]:
# Training
for stock, weird_columns in training_weird_columns_dict.items():
    if weird_columns:
        training_feature_df[stock] = handle_weird_data_with_knn_imputer(training_feature_df[stock], weird_columns)

# Testing
for stock, weird_columns in testing_weird_columns_dict.items():
    if weird_columns:
        testing_feature_df[stock] = handle_weird_data_with_knn_imputer(testing_feature_df[stock], weird_columns)

# Check the data again
for stock, stock_data in training_feature_df.items():
    if check_weird_data(stock_data):
        print(f"The {stock} stock price data still contains weird data in training data")

for stock, stock_data in testing_feature_df.items():
    if check_weird_data(stock_data):
        print(f"The {stock} stock price data still contains weird data in testing data")


In [97]:
# Clear weird_columns_dict, only clear the value, not the key
for stock in stock_list:
    training_weird_columns_dict[stock] = []
    testing_weird_columns_dict[stock] = []

In [98]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

        date  AAPL_exponential_moving_average_5  AAPL_exponential_moving_average_10  AAPL_exponential_moving_average_21  AAPL_downside_deviation_5  AAPL_downside_deviation_10  AAPL_downside_deviation_21  AAPL_sortino_ratio_5  AAPL_sortino_ratio_10  AAPL_sortino_ratio_21  AAPL_mom12m  AAPL_mom1m  AAPL_mom6m  AAPL_mvel1  AAPL_rvol_21d  AAPL_chcsho_1m  AAPL_chcsho_3m  AAPL_chcsho_6m  AAPL_ami_126d  AAPL_age  AAPL_chmom  AAPL_maxret   AAPL_dolvol  AAPL_beta_60m  AAPL_zero_trades_1m  AAPL_zero_trades_6m  AAPL_zero_trades_12m  AAPL_betabab_1260d  AAPL_earnings_variability  AAPL_ocfq_saleq_std  AAPL_turnover_126d  AAPL_prc_highprc_252d  AAPL_resff3_6_1  AAPL_seas_1_1na  AAPL_at_turnover      T_30  AAPL_VXN_LogDiff_EWMAhl63   CPI_ROC  T10Y2Y_EWMAhl10  T10Y2Y_LogDiff_EWMAhl21  DGS2_LogDiff_EWMAhl21  M2_RealGrowth  NFCI_LogDiff_EWMAhl21  USEPUINDXD_EWMAhl21  UMCSENT
0 2016-07-01                          23.972500                           23.972500                           23.972500            

#### Feature 43: New Privately-Owned Housing Units Started: Total Units  (HOUST)

In [99]:
def get_HOUST():
    HOUST = pd.read_excel(f"{DATA_DIR}/Monetary and Market Condition/HOUST.xlsx")
    target_cols = ['observation_date', 'HOUST']
    new_cols = HOUST[target_cols].copy()

    # Rename the columns
    new_cols.rename(columns={target_cols[1]: 'HOUST'}, inplace=True)

    # Parse the date column
    new_cols[target_cols[0]] = parse_dates(new_cols[target_cols[0]])

    return new_cols

# Training
for stock, stock_data in training_stock_df.items():
    new_cols = get_HOUST()
    substring = 'HOUST'
    date_substring = 'observation_date'

    # Merge the data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    # Backfill the missing values
    training_feature_df[stock][f'{substring}'] = training_feature_df[stock][f'{substring}'].bfill()

    if check_weird_data(training_feature_df[stock][f'{substring}']):
        print(f"Feature f'{substring}' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{substring}')

# Testing
for stock, stock_data in testing_stock_df.items():
    new_cols = get_HOUST()
    substring = 'HOUST'
    date_substring = 'observation_date'

    # Merge the data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    # Backfill the missing values
    testing_feature_df[stock][f'{substring}'] = testing_feature_df[stock][f'{substring}'].bfill()
    
    if check_weird_data(testing_feature_df[stock][f'{substring}']):
        print(f"Feature f'{substring}' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{substring}')

Feature f'HOUST' contains weird data in training data
Feature f'HOUST' contains weird data in training data
Feature f'HOUST' contains weird data in training data
Feature f'HOUST' contains weird data in training data
Feature f'HOUST' contains weird data in training data
Feature f'HOUST' contains weird data in training data
Feature f'HOUST' contains weird data in training data
Feature f'HOUST' contains weird data in training data
Feature f'HOUST' contains weird data in training data
Feature f'HOUST' contains weird data in testing data
Feature f'HOUST' contains weird data in testing data
Feature f'HOUST' contains weird data in testing data
Feature f'HOUST' contains weird data in testing data
Feature f'HOUST' contains weird data in testing data
Feature f'HOUST' contains weird data in testing data
Feature f'HOUST' contains weird data in testing data
Feature f'HOUST' contains weird data in testing data
Feature f'HOUST' contains weird data in testing data


In [100]:
# Training
for stock, weird_columns in training_weird_columns_dict.items():
    if weird_columns:
        training_feature_df[stock] = handle_weird_data_with_knn_imputer(training_feature_df[stock], weird_columns)

# Testing
for stock, weird_columns in testing_weird_columns_dict.items():
    if weird_columns:
        testing_feature_df[stock] = handle_weird_data_with_knn_imputer(testing_feature_df[stock], weird_columns)

# Check the data again
for stock, stock_data in training_feature_df.items():
    if check_weird_data(stock_data):
        print(f"The {stock} stock price data still contains weird data in training data")

for stock, stock_data in testing_feature_df.items():
    if check_weird_data(stock_data):
        print(f"The {stock} stock price data still contains weird data in testing data")


In [101]:
# Clear weird_columns_dict, only clear the value, not the key
for stock in stock_list:
    training_weird_columns_dict[stock] = []
    testing_weird_columns_dict[stock] = []

In [102]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

        date  AAPL_exponential_moving_average_5  AAPL_exponential_moving_average_10  AAPL_exponential_moving_average_21  AAPL_downside_deviation_5  AAPL_downside_deviation_10  AAPL_downside_deviation_21  AAPL_sortino_ratio_5  AAPL_sortino_ratio_10  AAPL_sortino_ratio_21  AAPL_mom12m  AAPL_mom1m  AAPL_mom6m  AAPL_mvel1  AAPL_rvol_21d  AAPL_chcsho_1m  AAPL_chcsho_3m  AAPL_chcsho_6m  AAPL_ami_126d  AAPL_age  AAPL_chmom  AAPL_maxret   AAPL_dolvol  AAPL_beta_60m  AAPL_zero_trades_1m  AAPL_zero_trades_6m  AAPL_zero_trades_12m  AAPL_betabab_1260d  AAPL_earnings_variability  AAPL_ocfq_saleq_std  AAPL_turnover_126d  AAPL_prc_highprc_252d  AAPL_resff3_6_1  AAPL_seas_1_1na  AAPL_at_turnover      T_30  AAPL_VXN_LogDiff_EWMAhl63   CPI_ROC  T10Y2Y_EWMAhl10  T10Y2Y_LogDiff_EWMAhl21  DGS2_LogDiff_EWMAhl21  M2_RealGrowth  NFCI_LogDiff_EWMAhl21  USEPUINDXD_EWMAhl21  UMCSENT   HOUST
0 2016-07-01                          23.972500                           23.972500                           23.972500    

#### Feature 44: Gross profits-to-assets  (gp_at)

In [103]:
def get_gp_at(stock):
    gp_at = pd.read_excel(f"{DATA_DIR}/global_factor/{stock}_global_factor.xlsx")
    target_cols = ['Day of last price observation (date)', 'Gross profits-to-assets (gp_at)']
    new_cols = gp_at[target_cols].copy()

    # Rename the columns
    new_cols.rename(columns={target_cols[1]: f'{stock}_gp_at'}, inplace=True)

    # Parse the date column
    new_cols[target_cols[0]] = parse_dates(new_cols[target_cols[0]])

    return new_cols

# Training
for stock, stock_data in training_stock_df.items():
    new_cols = get_gp_at(stock)
    substring = '_gp_at'
    date_substring = 'Day of last price observation (date)'

    # Merge the data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    # Backfill the missing values
    training_feature_df[stock][f'{stock}{substring}'] = training_feature_df[stock][f'{stock}{substring}'].bfill()

    if check_weird_data(training_feature_df[stock][f'{stock}{substring}']):
        print(f"Feature f'{stock}{substring}' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}{substring}')

# Testing
for stock, stock_data in testing_stock_df.items():
    new_cols = get_gp_at(stock)
    substring = '_gp_at'
    date_substring = 'Day of last price observation (date)'

    # Merge the data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    # Backfill the missing values
    testing_feature_df[stock][f'{stock}{substring}'] = testing_feature_df[stock][f'{stock}{substring}'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{stock}{substring}']):
        print(f"Feature f'{stock}{substring}' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}{substring}')

In [104]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

        date  AAPL_exponential_moving_average_5  AAPL_exponential_moving_average_10  AAPL_exponential_moving_average_21  AAPL_downside_deviation_5  AAPL_downside_deviation_10  AAPL_downside_deviation_21  AAPL_sortino_ratio_5  AAPL_sortino_ratio_10  AAPL_sortino_ratio_21  AAPL_mom12m  AAPL_mom1m  AAPL_mom6m  AAPL_mvel1  AAPL_rvol_21d  AAPL_chcsho_1m  AAPL_chcsho_3m  AAPL_chcsho_6m  AAPL_ami_126d  AAPL_age  AAPL_chmom  AAPL_maxret   AAPL_dolvol  AAPL_beta_60m  AAPL_zero_trades_1m  AAPL_zero_trades_6m  AAPL_zero_trades_12m  AAPL_betabab_1260d  AAPL_earnings_variability  AAPL_ocfq_saleq_std  AAPL_turnover_126d  AAPL_prc_highprc_252d  AAPL_resff3_6_1  AAPL_seas_1_1na  AAPL_at_turnover      T_30  AAPL_VXN_LogDiff_EWMAhl63   CPI_ROC  T10Y2Y_EWMAhl10  T10Y2Y_LogDiff_EWMAhl21  DGS2_LogDiff_EWMAhl21  M2_RealGrowth  NFCI_LogDiff_EWMAhl21  USEPUINDXD_EWMAhl21  UMCSENT   HOUST  AAPL_gp_at
0 2016-07-01                          23.972500                           23.972500                           2

#### Feature 45: Cash-based operating profits-tolagged book assets (cop_atl1)

In [105]:
def cop_atl1(stock):
    cop_atl1 = pd.read_excel(f"{DATA_DIR}/global_factor/{stock}_global_factor.xlsx")
    target_cols = ['Day of last price observation (date)', 'Cash-based operating profits-to-lagged book assets (cop_atl1)']
    new_cols = cop_atl1[target_cols].copy()

    # Rename the columns
    new_cols.rename(columns={target_cols[1]: f'{stock}_cop_atl1'}, inplace=True)

    # Parse the date column
    new_cols[target_cols[0]] = parse_dates(new_cols[target_cols[0]])

    return new_cols

# Training
for stock, stock_data in training_stock_df.items():
    new_cols = cop_atl1(stock)
    substring = '_cop_atl1'
    date_substring = 'Day of last price observation (date)'

    # Merge the data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    # Backfill the missing values
    training_feature_df[stock][f'{stock}{substring}'] = training_feature_df[stock][f'{stock}{substring}'].bfill()

    if check_weird_data(training_feature_df[stock][f'{stock}{substring}']):
        print(f"Feature f'{stock}{substring}' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}{substring}')

# Testing
for stock, stock_data in testing_stock_df.items():
    new_cols = cop_atl1(stock)
    substring = '_cop_atl1'
    date_substring = 'Day of last price observation (date)'

    # Merge the data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    # Backfill the missing values
    testing_feature_df[stock][f'{stock}{substring}'] = testing_feature_df[stock][f'{stock}{substring}'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{stock}{substring}']):
        print(f"Feature f'{stock}{substring}' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}{substring}')

In [106]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

        date  AAPL_exponential_moving_average_5  AAPL_exponential_moving_average_10  AAPL_exponential_moving_average_21  AAPL_downside_deviation_5  AAPL_downside_deviation_10  AAPL_downside_deviation_21  AAPL_sortino_ratio_5  AAPL_sortino_ratio_10  AAPL_sortino_ratio_21  AAPL_mom12m  AAPL_mom1m  AAPL_mom6m  AAPL_mvel1  AAPL_rvol_21d  AAPL_chcsho_1m  AAPL_chcsho_3m  AAPL_chcsho_6m  AAPL_ami_126d  AAPL_age  AAPL_chmom  AAPL_maxret   AAPL_dolvol  AAPL_beta_60m  AAPL_zero_trades_1m  AAPL_zero_trades_6m  AAPL_zero_trades_12m  AAPL_betabab_1260d  AAPL_earnings_variability  AAPL_ocfq_saleq_std  AAPL_turnover_126d  AAPL_prc_highprc_252d  AAPL_resff3_6_1  AAPL_seas_1_1na  AAPL_at_turnover      T_30  AAPL_VXN_LogDiff_EWMAhl63   CPI_ROC  T10Y2Y_EWMAhl10  T10Y2Y_LogDiff_EWMAhl21  DGS2_LogDiff_EWMAhl21  M2_RealGrowth  NFCI_LogDiff_EWMAhl21  USEPUINDXD_EWMAhl21  UMCSENT   HOUST  AAPL_gp_at  AAPL_cop_atl1
0 2016-07-01                          23.972500                           23.972500             

#### Feature 46: Operating profits-to-lagged book assets (op_atl1)

In [107]:
def get_op_atlt1(stock):
    op_atlt1 = pd.read_excel(f"{DATA_DIR}/global_factor/{stock}_global_factor.xlsx")
    target_cols = ['Day of last price observation (date)', 'Operating profits-to-lagged book assets (op_atl1)']
    new_cols = op_atlt1[target_cols].copy()

    # Rename the columns
    new_cols.rename(columns={target_cols[1]: f'{stock}_op_atlt1'}, inplace=True)

    # Parse the date column
    new_cols[target_cols[0]] = parse_dates(new_cols[target_cols[0]])

    return new_cols

# Training
for stock, stock_data in training_stock_df.items():
    new_cols = get_op_atlt1(stock)
    substring = '_op_atlt1'
    date_substring = 'Day of last price observation (date)'

    # Merge the data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    # Backfill the missing values
    training_feature_df[stock][f'{stock}{substring}'] = training_feature_df[stock][f'{stock}{substring}'].bfill()

    if check_weird_data(training_feature_df[stock][f'{stock}{substring}']):
        print(f"Feature f'{stock}{substring}' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}{substring}')

# Testing
for stock, stock_data in testing_stock_df.items():
    new_cols = get_op_atlt1(stock)
    substring = '_op_atlt1'
    date_substring = 'Day of last price observation (date)'

    # Merge the data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    # Backfill the missing values
    testing_feature_df[stock][f'{stock}{substring}'] = testing_feature_df[stock][f'{stock}{substring}'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{stock}{substring}']):
        print(f"Feature f'{stock}{substring}' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}{substring}')

In [108]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

        date  AAPL_exponential_moving_average_5  AAPL_exponential_moving_average_10  AAPL_exponential_moving_average_21  AAPL_downside_deviation_5  AAPL_downside_deviation_10  AAPL_downside_deviation_21  AAPL_sortino_ratio_5  AAPL_sortino_ratio_10  AAPL_sortino_ratio_21  AAPL_mom12m  AAPL_mom1m  AAPL_mom6m  AAPL_mvel1  AAPL_rvol_21d  AAPL_chcsho_1m  AAPL_chcsho_3m  AAPL_chcsho_6m  AAPL_ami_126d  AAPL_age  AAPL_chmom  AAPL_maxret   AAPL_dolvol  AAPL_beta_60m  AAPL_zero_trades_1m  AAPL_zero_trades_6m  AAPL_zero_trades_12m  AAPL_betabab_1260d  AAPL_earnings_variability  AAPL_ocfq_saleq_std  AAPL_turnover_126d  AAPL_prc_highprc_252d  AAPL_resff3_6_1  AAPL_seas_1_1na  AAPL_at_turnover      T_30  AAPL_VXN_LogDiff_EWMAhl63   CPI_ROC  T10Y2Y_EWMAhl10  T10Y2Y_LogDiff_EWMAhl21  DGS2_LogDiff_EWMAhl21  M2_RealGrowth  NFCI_LogDiff_EWMAhl21  USEPUINDXD_EWMAhl21  UMCSENT   HOUST  AAPL_gp_at  AAPL_cop_atl1  AAPL_op_atlt1
0 2016-07-01                          23.972500                           23.9725

#### Feature 47: Quarterly return on assets (niq_at)

In [109]:
def get_niq_at(stock):
    niq_at = pd.read_excel(f"{DATA_DIR}/global_factor/{stock}_global_factor.xlsx")
    target_cols = ['Day of last price observation (date)', 'Quarterly return on assets (niq_at)']
    new_cols = niq_at[target_cols].copy()

    # Rename the columns
    new_cols.rename(columns={target_cols[1]: f'{stock}_niq_at'}, inplace=True)

    # Parse the date column
    new_cols[target_cols[0]] = parse_dates(new_cols[target_cols[0]])

    return new_cols

# Training
for stock, stock_data in training_stock_df.items():
    new_cols = get_niq_at(stock)
    substring = '_niq_at'
    date_substring = 'Day of last price observation (date)'

    # Merge the data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    # Backfill the missing values
    training_feature_df[stock][f'{stock}{substring}'] = training_feature_df[stock][f'{stock}{substring}'].bfill()

    if check_weird_data(training_feature_df[stock][f'{stock}{substring}']):
        print(f"Feature f'{stock}{substring}' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}{substring}')

# Testing
for stock, stock_data in testing_stock_df.items():
    new_cols = get_niq_at(stock)
    substring = '_niq_at'
    date_substring = 'Day of last price observation (date)'

    # Merge the data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    # Backfill the missing values
    testing_feature_df[stock][f'{stock}{substring}'] = testing_feature_df[stock][f'{stock}{substring}'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{stock}{substring}']):
        print(f"Feature f'{stock}{substring}' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}{substring}')

In [110]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

        date  AAPL_exponential_moving_average_5  AAPL_exponential_moving_average_10  AAPL_exponential_moving_average_21  AAPL_downside_deviation_5  AAPL_downside_deviation_10  AAPL_downside_deviation_21  AAPL_sortino_ratio_5  AAPL_sortino_ratio_10  AAPL_sortino_ratio_21  AAPL_mom12m  AAPL_mom1m  AAPL_mom6m  AAPL_mvel1  AAPL_rvol_21d  AAPL_chcsho_1m  AAPL_chcsho_3m  AAPL_chcsho_6m  AAPL_ami_126d  AAPL_age  AAPL_chmom  AAPL_maxret   AAPL_dolvol  AAPL_beta_60m  AAPL_zero_trades_1m  AAPL_zero_trades_6m  AAPL_zero_trades_12m  AAPL_betabab_1260d  AAPL_earnings_variability  AAPL_ocfq_saleq_std  AAPL_turnover_126d  AAPL_prc_highprc_252d  AAPL_resff3_6_1  AAPL_seas_1_1na  AAPL_at_turnover      T_30  AAPL_VXN_LogDiff_EWMAhl63   CPI_ROC  T10Y2Y_EWMAhl10  T10Y2Y_LogDiff_EWMAhl21  DGS2_LogDiff_EWMAhl21  M2_RealGrowth  NFCI_LogDiff_EWMAhl21  USEPUINDXD_EWMAhl21  UMCSENT   HOUST  AAPL_gp_at  AAPL_cop_atl1  AAPL_op_atlt1  AAPL_niq_at
0 2016-07-01                          23.972500                     

#### Feature 48: Quality minus Junk: Composite (qmj)

In [111]:
def get_qmj(stock):
    qmj = pd.read_excel(f"{DATA_DIR}/global_factor/{stock}_global_factor.xlsx")
    target_cols = ['Day of last price observation (date)', 'Quality minus Junk: Composite (qmj)']
    new_cols = qmj[target_cols].copy()

    # Rename the columns
    new_cols.rename(columns={target_cols[1]: f'{stock}_qmj'}, inplace=True)

    # Parse the date column
    new_cols[target_cols[0]] = parse_dates(new_cols[target_cols[0]])

    return new_cols

# Training
for stock, stock_data in training_stock_df.items():
    new_cols = get_qmj(stock)
    substring = '_qmj'
    date_substring = 'Day of last price observation (date)'

    # Merge the data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    # Backfill the missing values
    training_feature_df[stock][f'{stock}{substring}'] = training_feature_df[stock][f'{stock}{substring}'].bfill()

    if check_weird_data(training_feature_df[stock][f'{stock}{substring}']):
        print(f"Feature f'{stock}{substring}' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}{substring}')

# Testing
for stock, stock_data in testing_stock_df.items():
    new_cols = get_qmj(stock)
    substring = '_qmj'
    date_substring = 'Day of last price observation (date)'

    # Merge the data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    # Backfill the missing values
    testing_feature_df[stock][f'{stock}{substring}'] = testing_feature_df[stock][f'{stock}{substring}'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{stock}{substring}']):
        print(f"Feature f'{stock}{substring}' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}{substring}')

In [112]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

        date  AAPL_exponential_moving_average_5  AAPL_exponential_moving_average_10  AAPL_exponential_moving_average_21  AAPL_downside_deviation_5  AAPL_downside_deviation_10  AAPL_downside_deviation_21  AAPL_sortino_ratio_5  AAPL_sortino_ratio_10  AAPL_sortino_ratio_21  AAPL_mom12m  AAPL_mom1m  AAPL_mom6m  AAPL_mvel1  AAPL_rvol_21d  AAPL_chcsho_1m  AAPL_chcsho_3m  AAPL_chcsho_6m  AAPL_ami_126d  AAPL_age  AAPL_chmom  AAPL_maxret   AAPL_dolvol  AAPL_beta_60m  AAPL_zero_trades_1m  AAPL_zero_trades_6m  AAPL_zero_trades_12m  AAPL_betabab_1260d  AAPL_earnings_variability  AAPL_ocfq_saleq_std  AAPL_turnover_126d  AAPL_prc_highprc_252d  AAPL_resff3_6_1  AAPL_seas_1_1na  AAPL_at_turnover      T_30  AAPL_VXN_LogDiff_EWMAhl63   CPI_ROC  T10Y2Y_EWMAhl10  T10Y2Y_LogDiff_EWMAhl21  DGS2_LogDiff_EWMAhl21  M2_RealGrowth  NFCI_LogDiff_EWMAhl21  USEPUINDXD_EWMAhl21  UMCSENT   HOUST  AAPL_gp_at  AAPL_cop_atl1  AAPL_op_atlt1  AAPL_niq_at  AAPL_qmj
0 2016-07-01                          23.972500           

#### Feature 49: Quality minus Junk: Growth (qmj_growth)

In [113]:
def get_qmj_growth(stock):
    qmj_growth = pd.read_excel(f"{DATA_DIR}/global_factor/{stock}_global_factor.xlsx")
    target_cols = ['Day of last price observation (date)', 'Quality minus Junk: Growth (qmj_growth)']
    new_cols = qmj_growth[target_cols].copy()

    # Rename the columns
    new_cols.rename(columns={target_cols[1]: f'{stock}_qmj_growth'}, inplace=True)

    # Parse the date column
    new_cols[target_cols[0]] = parse_dates(new_cols[target_cols[0]])

    return new_cols

# Training
for stock, stock_data in training_stock_df.items():
    new_cols = get_qmj_growth(stock)
    substring = '_qmj_growth'
    date_substring = 'Day of last price observation (date)'

    # Merge the data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    # Backfill the missing values
    training_feature_df[stock][f'{stock}{substring}'] = training_feature_df[stock][f'{stock}{substring}'].bfill()

    if check_weird_data(training_feature_df[stock][f'{stock}{substring}']):
        print(f"Feature f'{stock}{substring}' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}{substring}')

# Testing
for stock, stock_data in testing_stock_df.items():
    new_cols = get_qmj_growth(stock)
    substring = '_qmj_growth'
    date_substring = 'Day of last price observation (date)'

    # Merge the data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    # Backfill the missing values
    testing_feature_df[stock][f'{stock}{substring}'] = testing_feature_df[stock][f'{stock}{substring}'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{stock}{substring}']):
        print(f"Feature f'{stock}{substring}' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}{substring}')

In [114]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

        date  AAPL_exponential_moving_average_5  AAPL_exponential_moving_average_10  AAPL_exponential_moving_average_21  AAPL_downside_deviation_5  AAPL_downside_deviation_10  AAPL_downside_deviation_21  AAPL_sortino_ratio_5  AAPL_sortino_ratio_10  AAPL_sortino_ratio_21  AAPL_mom12m  AAPL_mom1m  AAPL_mom6m  AAPL_mvel1  AAPL_rvol_21d  AAPL_chcsho_1m  AAPL_chcsho_3m  AAPL_chcsho_6m  AAPL_ami_126d  AAPL_age  AAPL_chmom  AAPL_maxret   AAPL_dolvol  AAPL_beta_60m  AAPL_zero_trades_1m  AAPL_zero_trades_6m  AAPL_zero_trades_12m  AAPL_betabab_1260d  AAPL_earnings_variability  AAPL_ocfq_saleq_std  AAPL_turnover_126d  AAPL_prc_highprc_252d  AAPL_resff3_6_1  AAPL_seas_1_1na  AAPL_at_turnover      T_30  AAPL_VXN_LogDiff_EWMAhl63   CPI_ROC  T10Y2Y_EWMAhl10  T10Y2Y_LogDiff_EWMAhl21  DGS2_LogDiff_EWMAhl21  M2_RealGrowth  NFCI_LogDiff_EWMAhl21  USEPUINDXD_EWMAhl21  UMCSENT   HOUST  AAPL_gp_at  AAPL_cop_atl1  AAPL_op_atlt1  AAPL_niq_at  AAPL_qmj  AAPL_qmj_growth
0 2016-07-01                          23.

#### Feature 50: Quality minus Junk: Safety (qmj_safety)

In [115]:
def get_qmj_safety(stock):
    qmj_safety = pd.read_excel(f"{DATA_DIR}/global_factor/{stock}_global_factor.xlsx")
    target_cols = ['Day of last price observation (date)', 'Quality minus Junk: Safety (qmj_safety)']
    new_cols = qmj_safety[target_cols].copy()

    # Rename the columns
    new_cols.rename(columns={target_cols[1]: f'{stock}_qmj_safety'}, inplace=True)

    # Parse the date column
    new_cols[target_cols[0]] = parse_dates(new_cols[target_cols[0]])

    return new_cols

# Training
for stock, stock_data in training_stock_df.items():
    new_cols = get_qmj_safety(stock)
    substring = '_qmj_safety'
    date_substring = 'Day of last price observation (date)'

    # Merge the data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    # Backfill the missing values
    training_feature_df[stock][f'{stock}{substring}'] = training_feature_df[stock][f'{stock}{substring}'].bfill()

    if check_weird_data(training_feature_df[stock][f'{stock}{substring}']):
        print(f"Feature f'{stock}{substring}' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}{substring}')

# Testing
for stock, stock_data in testing_stock_df.items():
    new_cols = get_qmj_safety(stock)
    substring = '_qmj_safety'
    date_substring = 'Day of last price observation (date)'

    # Merge the data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    # Backfill the missing values
    testing_feature_df[stock][f'{stock}{substring}'] = testing_feature_df[stock][f'{stock}{substring}'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{stock}{substring}']):
        print(f"Feature f'{stock}{substring}' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}{substring}')

In [116]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

        date  AAPL_exponential_moving_average_5  AAPL_exponential_moving_average_10  AAPL_exponential_moving_average_21  AAPL_downside_deviation_5  AAPL_downside_deviation_10  AAPL_downside_deviation_21  AAPL_sortino_ratio_5  AAPL_sortino_ratio_10  AAPL_sortino_ratio_21  AAPL_mom12m  AAPL_mom1m  AAPL_mom6m  AAPL_mvel1  AAPL_rvol_21d  AAPL_chcsho_1m  AAPL_chcsho_3m  AAPL_chcsho_6m  AAPL_ami_126d  AAPL_age  AAPL_chmom  AAPL_maxret   AAPL_dolvol  AAPL_beta_60m  AAPL_zero_trades_1m  AAPL_zero_trades_6m  AAPL_zero_trades_12m  AAPL_betabab_1260d  AAPL_earnings_variability  AAPL_ocfq_saleq_std  AAPL_turnover_126d  AAPL_prc_highprc_252d  AAPL_resff3_6_1  AAPL_seas_1_1na  AAPL_at_turnover      T_30  AAPL_VXN_LogDiff_EWMAhl63   CPI_ROC  T10Y2Y_EWMAhl10  T10Y2Y_LogDiff_EWMAhl21  DGS2_LogDiff_EWMAhl21  M2_RealGrowth  NFCI_LogDiff_EWMAhl21  USEPUINDXD_EWMAhl21  UMCSENT   HOUST  AAPL_gp_at  AAPL_cop_atl1  AAPL_op_atlt1  AAPL_niq_at  AAPL_qmj  AAPL_qmj_growth  AAPL_qmj_safety
0 2016-07-01            

#### Feature 51: Operating leverage (opex_at)

In [117]:
def get_opex_at(stock):
    opex_at = pd.read_excel(f"{DATA_DIR}/global_factor/{stock}_global_factor.xlsx")
    target_cols = ['Day of last price observation (date)', 'Operating leverage (opex_at)']
    new_cols = opex_at[target_cols].copy()

    # Rename the columns
    new_cols.rename(columns={target_cols[1]: f'{stock}_opex_at'}, inplace=True)

    # Parse the date column
    new_cols[target_cols[0]] = parse_dates(new_cols[target_cols[0]])

    return new_cols

# Training
for stock, stock_data in training_stock_df.items():
    new_cols = get_opex_at(stock)
    substring = '_opex_at'
    date_substring = 'Day of last price observation (date)'

    # Merge the data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    # Backfill the missing values
    training_feature_df[stock][f'{stock}{substring}'] = training_feature_df[stock][f'{stock}{substring}'].bfill()

    if check_weird_data(training_feature_df[stock][f'{stock}{substring}']):
        print(f"Feature f'{stock}{substring}' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}{substring}')

# Testing
for stock, stock_data in testing_stock_df.items():
    new_cols = get_opex_at(stock)
    substring = '_opex_at'
    date_substring = 'Day of last price observation (date)'

    # Merge the data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    # Backfill the missing values
    testing_feature_df[stock][f'{stock}{substring}'] = testing_feature_df[stock][f'{stock}{substring}'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{stock}{substring}']):
        print(f"Feature f'{stock}{substring}' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}{substring}')

In [118]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

        date  AAPL_exponential_moving_average_5  AAPL_exponential_moving_average_10  AAPL_exponential_moving_average_21  AAPL_downside_deviation_5  AAPL_downside_deviation_10  AAPL_downside_deviation_21  AAPL_sortino_ratio_5  AAPL_sortino_ratio_10  AAPL_sortino_ratio_21  AAPL_mom12m  AAPL_mom1m  AAPL_mom6m  AAPL_mvel1  AAPL_rvol_21d  AAPL_chcsho_1m  AAPL_chcsho_3m  AAPL_chcsho_6m  AAPL_ami_126d  AAPL_age  AAPL_chmom  AAPL_maxret   AAPL_dolvol  AAPL_beta_60m  AAPL_zero_trades_1m  AAPL_zero_trades_6m  AAPL_zero_trades_12m  AAPL_betabab_1260d  AAPL_earnings_variability  AAPL_ocfq_saleq_std  AAPL_turnover_126d  AAPL_prc_highprc_252d  AAPL_resff3_6_1  AAPL_seas_1_1na  AAPL_at_turnover      T_30  AAPL_VXN_LogDiff_EWMAhl63   CPI_ROC  T10Y2Y_EWMAhl10  T10Y2Y_LogDiff_EWMAhl21  DGS2_LogDiff_EWMAhl21  M2_RealGrowth  NFCI_LogDiff_EWMAhl21  USEPUINDXD_EWMAhl21  UMCSENT   HOUST  AAPL_gp_at  AAPL_cop_atl1  AAPL_op_atlt1  AAPL_niq_at  AAPL_qmj  AAPL_qmj_growth  AAPL_qmj_safety  AAPL_opex_at
0 2016-07-

#### Feature 52: Number of consecutive quarters with earnings increases (ni_inc8q)

In [119]:
def get_ni_inc8q(stock):
    ni_inc8q = pd.read_excel(f"{DATA_DIR}/global_factor/{stock}_global_factor.xlsx")
    target_cols = ['Day of last price observation (date)', 'Number of consecutive quarters with earnings increases (ni_inc8']
    new_cols = ni_inc8q[target_cols].copy()

    # Rename the columns
    new_cols.rename(columns={target_cols[1]: f'{stock}_ni_inc8q'}, inplace=True)

    # Parse the date column
    new_cols[target_cols[0]] = parse_dates(new_cols[target_cols[0]])

    return new_cols

# Training
for stock, stock_data in training_stock_df.items():
    new_cols = get_ni_inc8q(stock)
    substring = '_ni_inc8q'
    date_substring = 'Day of last price observation (date)'

    # Merge the data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    # Backfill the missing values
    training_feature_df[stock][f'{stock}{substring}'] = training_feature_df[stock][f'{stock}{substring}'].bfill()

    if check_weird_data(training_feature_df[stock][f'{stock}{substring}']):
        print(f"Feature f'{stock}{substring}' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}{substring}')

# Testing
for stock, stock_data in testing_stock_df.items():
    new_cols = get_ni_inc8q(stock)
    substring = '_ni_inc8q'
    date_substring = 'Day of last price observation (date)'

    # Merge the data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    # Backfill the missing values
    testing_feature_df[stock][f'{stock}{substring}'] = testing_feature_df[stock][f'{stock}{substring}'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{stock}{substring}']):
        print(f"Feature f'{stock}{substring}' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}{substring}')


In [120]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

        date  AAPL_exponential_moving_average_5  AAPL_exponential_moving_average_10  AAPL_exponential_moving_average_21  AAPL_downside_deviation_5  AAPL_downside_deviation_10  AAPL_downside_deviation_21  AAPL_sortino_ratio_5  AAPL_sortino_ratio_10  AAPL_sortino_ratio_21  AAPL_mom12m  AAPL_mom1m  AAPL_mom6m  AAPL_mvel1  AAPL_rvol_21d  AAPL_chcsho_1m  AAPL_chcsho_3m  AAPL_chcsho_6m  AAPL_ami_126d  AAPL_age  AAPL_chmom  AAPL_maxret   AAPL_dolvol  AAPL_beta_60m  AAPL_zero_trades_1m  AAPL_zero_trades_6m  AAPL_zero_trades_12m  AAPL_betabab_1260d  AAPL_earnings_variability  AAPL_ocfq_saleq_std  AAPL_turnover_126d  AAPL_prc_highprc_252d  AAPL_resff3_6_1  AAPL_seas_1_1na  AAPL_at_turnover      T_30  AAPL_VXN_LogDiff_EWMAhl63   CPI_ROC  T10Y2Y_EWMAhl10  T10Y2Y_LogDiff_EWMAhl21  DGS2_LogDiff_EWMAhl21  M2_RealGrowth  NFCI_LogDiff_EWMAhl21  USEPUINDXD_EWMAhl21  UMCSENT   HOUST  AAPL_gp_at  AAPL_cop_atl1  AAPL_op_atlt1  AAPL_niq_at  AAPL_qmj  AAPL_qmj_growth  AAPL_qmj_safety  AAPL_opex_at  AAPL_ni_i

Save the training set and test set to excel files

In [122]:
# Save the training and testing data to excel files
for stock in stock_list:
    # If the directory does not exist, create it
    os.makedirs(f"{DATA_DIR}/training_data", exist_ok=True)
    os.makedirs(f"{DATA_DIR}/testing_data", exist_ok=True)

    # Save the training and testing data to excel files
    training_feature_df[stock].to_excel(f"{DATA_DIR}/training_data/training_feature_{stock}.xlsx", index=False)
    testing_feature_df[stock].to_excel(f"{DATA_DIR}/testing_data/testing_feature_{stock}.xlsx", index=False)